<a href="https://colab.research.google.com/github/Vaccine-studies/Safety-of-the-Herpes-Zoster-Vaccine-in-the-Older-New-Zealand-Population/blob/main/Copy_of_MoH_R3_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Load package

In [ ]:
import pandas as pd
pd.set_option('precision', 4)

pd.options.mode.chained_assignment = None  # default='warn'

import numpy as np

from datetime import date, timedelta

import statsmodels.api as sm
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

from pandas_profiling import ProfileReport

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dir="/content/drive/MyDrive/Fun PhD journey/Endeavour/"

In [ ]:
#missing value/percentage check
def missingvalue(data):
    columns = data.columns
    types=data.dtypes
    percent_missing = data.isnull().sum() * 100 / len(data)
    missing_value_df = pd.DataFrame({'column_name': columns,'column_type':types,'percent_missing': percent_missing})
    missing_value_df=missing_value_df[missing_value_df.percent_missing>0]
    return(missing_value_df.sort_values(by=['percent_missing'],ascending=False))


def dataprofile(df,removemissing=False):
  ##Capitalize column names
  df.columns=df.columns.str.upper()
  to_drop=[]
  temp=[0,0]
  ##create missing value profile
  missingdata=missingvalue(df)
  if removemissing:
    missingcol=missingdata[missingdata.percent_missing>=50].column_name
    df.drop(missingcol,inplace=True,axis=1)
  for i in df.columns:
    if(df[i].dtype=='object'):
      df[i]=df[i].str.strip()
    try:
      ##print missing value >20%
      missingp=missingdata.loc[i,'percent_missing']
      if(missingp>20):
        print(i + " has missing value percentage of" + str(missingp))
    except:
      pass
      ##print single value features
    if(df[i].nunique()==1):
      print(i + " has been dropped due to there is single value in this feature")
      to_drop.append(i)
    elif(df[i].nunique()==df.shape[0]):
      print(i + " may be id as values are totally unique")
    elif(df[i].nunique()==2):
      # df[i]=df[i].astype('bool')
      print(i + " IS A BINARY FEATURE")
    elif(df[i].nunique()==temp[1]):
      print(i + " may be projected correlated with " +str(temp[0]))
    temp=[i,df[i].nunique()]
  print('-----------------------------------------------------------------')
  return df.drop(to_drop,axis=1)


def removeduplicate(df,ids=["EVENT_ID"],top=20):
  df['New']=0
  df['New']=df.groupby(ids).cumcount()+1
  print(df['New'].value_counts().head(20))
  ids.append('New')
  new_df=df.set_index(ids).unstack('New').sort_index(axis=1 , level=1)
  new_df.columns=new_df.columns.map('{0[0]}{0[1]}'.format) 
  new_df=new_df.iloc[:,:top]
  return new_df

def psize(df):
  print(df.shape)

#Load file

In [ ]:
import os
# from pandas_profiling import ProfileReport
folder='MOH-DataServices_Request 2'
folder=dir+folder
df=[]
for file in sorted(os.listdir(folder)):
    if file.endswith(".txt"):
        print(file)
        temp_df=pd.read_csv(os.path.join(folder, file),sep='|')
        df.append(temp_df)
        # prof = ProfileReport(temp_df)
        # prof.to_file(output_file=folder+'/output_' +str(file) +'.html')

MOH-DataServices_cas2207.txt
MOH-DataServices_diags_control_2018.txt
MOH-DataServices_diags_control_2019.txt
MOH-DataServices_diags_control_2020.txt
MOH-DataServices_diags_control_2021.txt
MOH-DataServices_diags_control_lookback_2018.txt
MOH-DataServices_diags_control_lookback_2019.txt
MOH-DataServices_diags_control_lookback_2020.txt
MOH-DataServices_diags_control_lookback_2021.txt
MOH-DataServices_diags_high_risk_2018.txt
MOH-DataServices_diags_high_risk_2019.txt
MOH-DataServices_diags_high_risk_2020.txt
MOH-DataServices_diags_high_risk_2021.txt
MOH-DataServices_diags_hr_lookback_2018.txt
MOH-DataServices_diags_hr_lookback_2019.txt
MOH-DataServices_diags_hr_lookback_2020.txt
MOH-DataServices_diags_hr_lookback_2021.txt
MOH-DataServices_events_control_2018.txt
MOH-DataServices_events_control_2019.txt
MOH-DataServices_events_control_2020.txt
MOH-DataServices_events_control_2021.txt
MOH-DataServices_events_control_lookback_2018.txt
MOH-DataServices_events_control_lookback_2019.txt
MOH-Dat

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
cas,dc18,dc19,dc20,dc21,dclb18,dclb19,dclb20,dclb21,dh18,dh19,dh20,dh21,dhlb18,dhlb19,dhlb20,dhlb21,ec18,ec19,ec20,ec21,eclb18,eclb19,eclb20,eclb21,eh18,eh19,eh20,eh21,ehlb18,ehlb19,ehlb20,ehlb21,mis=df[:]

In [ ]:
pusdig_ctr=pd.concat([dc18,dc19,dc20,dc21])
pusdig_ctrlb=pd.concat([dclb18,dclb19,dclb20,dclb21])
pusdig_hig=pd.concat([dh18,dh19,dh20,dh21])
pusdig_higlb=pd.concat([dhlb18,dhlb19,dhlb20,dhlb21])
puseve_ctr=pd.concat([ec18,ec19,ec20,ec21])
puseve_ctrlb=pd.concat([eclb18,eclb19,eclb20,eclb21])
puseve_hig=pd.concat([eh18,eh19,eh20,eh21])
puseve_higlb=pd.concat([ehlb18,ehlb19,ehlb20,ehlb21])

In [ ]:
#rename specific column names
puseve_ctr.rename(columns = {'DHB_2015':'DHBDOM'}, inplace = True)
puseve_ctrlb.rename(columns = {'DHB_2015':'DHBDOM'}, inplace = True)
puseve_hig.rename(columns = {'DHB_2015':'DHBDOM'}, inplace = True)
puseve_higlb.rename(columns = {'DHB_2015':'DHBDOM'}, inplace = True)

can=dataprofile(cas) 
dem=dataprofile(mis) 
pusdig_ctr=dataprofile(pusdig_ctr)
pusdig_hig=dataprofile(pusdig_hig)
puseve_ctr=dataprofile(puseve_ctr)
puseve_hig=dataprofile(puseve_hig)
pusdig_ctrlb=dataprofile(pusdig_ctrlb)
pusdig_higlb=dataprofile(pusdig_higlb)
puseve_ctrlb=dataprofile(puseve_ctrlb)
puseve_higlb=dataprofile(puseve_higlb)

ENC_NHI may be projected correlated with MASTER_ENC
GENDER IS A BINARY FEATURE
ETHNICG1 may be projected correlated with ETHNICGP
ETHNICG3 has been dropped due to there is single value in this feature
DEP13 may be projected correlated with DEP06
DEP18 may be projected correlated with DEP13
-----------------------------------------------------------------
UNIQUE_ID may be id as values are totally unique
DOD has missing value percentage of94.09810280075943
ETHNIC_CODE_1 may be projected correlated with PRIORITY_ETHNIC_CODE
ETHNIC_CODE_2 has missing value percentage of95.93155392951677
ETHNIC_CODE_2 may be projected correlated with ETHNIC_CODE_1
ETHNIC_CODE_3 has missing value percentage of99.88151166174697
DEP18 may be projected correlated with DEP13
-----------------------------------------------------------------
CLIN_SYS has been dropped due to there is single value in this feature
SUBMITTED_SYSTEM_ID IS A BINARY FEATURE
----------------------------------------------------------------

In [ ]:
##Drop duplicates
dem=dem.drop_duplicates(['MASTER_ENC'])

domcile=pd.read_csv('/content/drive/MyDrive/Fun PhD journey/Endeavour/domicile_code_table_census_v3_2013_0.csv')
morph=pd.read_csv('/content/drive/MyDrive/Fun PhD journey/Endeavour/ICD-10-v8-morph.csv',index_col='ICD')
imd=pd.read_csv('/content/drive/MyDrive/Fun PhD journey/Endeavour/immuneodeficiencies.txt',delimiter='|')

DHB_CODE=['011','021','022','023','031','042','047','051','071','061','081','082','091','092','093','101','111','121','123','160','999','XXX']
DHB_DESCR=["NORTHLAND","WAITEMATA","AUCKLAND","COUNTIES MANUKAU","WAIKATO","LAKES","BAY OF PLENTY","TAIRAWHITI","TARANAKI","HAWKE'S BAY","MIDCENTRAL","WHANGANUI","CAPITAL AND COAST",
           "HUTT","WAIRARAPA","NELSON MARLBOROUGH","WEST COAST","CANTERBURY","SOUTH CANTERBURY","SOUTHERN","OVERSEAS","UNKNOWN"]

DHB_Mapping=pd.DataFrame(list(zip(DHB_CODE, DHB_DESCR)),columns =['DHBDOM', 'LOCATION'])

cat_list=['AGE','GENDER','ETHNICGP','DEP06','DEP13','DEP06_Q','DEP13_Q','LOCATION']

disease_list=['COPD','DIAB','GOUT','CERE','CHRO','ISCH','SERO','PSOE','LUPU','BOWE','PSOR','ANKY','LIVE','CANC','WHOC','APPE','HERN','DIVE','FEMF','CHOC','PAND','SEPS','HAEM','RENC','BURN','LIPO','EPIS']
##list of people who got immuse system problem
imd_list=imd.MASTER_ENCRYPTED_HCU_ID.values
##different cancers list
can_list=[8000,8010,8046,8070,8077,8130,8140,8160,8240,8260,8310,8380,8480,8500,8720,8721,8742,8743,9050,9061,9085,9220,9440,9670,9680,9698,9699,6732,9945,9983]

table_list=['DIAG_YEAR','AGE','GENDER','ETHNICGP_Lvl1','DEP06_Q','DEP13_Q','DEP18_Q','IMD','LOCATION']

#Analyze - Table1

In [ ]:
##Return Primary Diagnose Only?
Type_A_only=True

high risk

In [ ]:
pusdig=pusdig_hig
print('Original pusdig size')
psize(pusdig)
if Type_A_only:
  pusdig=pusdig[pusdig.DIAG_TYP=='A']

print('Type A diagnose pusdig size')
psize(pusdig)

puseve=puseve_hig.drop_duplicates(['MASTER_ENC','EVENT_ID'])

##Get EVENT_ID for different disease
COPD_list=list(pusdig[pusdig.CLIN_CD.str.contains('J40') | pusdig.CLIN_CD.str.contains('J41') | pusdig.CLIN_CD.str.contains('J42') | pusdig.CLIN_CD.str.contains('J43') | pusdig.CLIN_CD.str.contains('J44')].EVENT_ID)
DIAB_list=list(pusdig[pusdig.CLIN_CD.str.contains('E10') | pusdig.CLIN_CD.str.contains('E11') | pusdig.CLIN_CD.str.contains('E12') | pusdig.CLIN_CD.str.contains('E13') | pusdig.CLIN_CD.str.contains('E14')].EVENT_ID)
GOUT_list=list(pusdig[pusdig.CLIN_CD.str.startswith('M10')].EVENT_ID)
CERE_list=list(pusdig[pusdig.CLIN_CD.str.startswith('I6')].EVENT_ID)
CHRO_list=list(pusdig[pusdig.CLIN_CD.str.startswith('N18')].EVENT_ID)
ISCH_list=list(pusdig[pusdig.CLIN_CD.str.contains('I20') | pusdig.CLIN_CD.str.contains('I21') | pusdig.CLIN_CD.str.contains('I22') | pusdig.CLIN_CD.str.contains('I23') | pusdig.CLIN_CD.str.contains('I24')| pusdig.CLIN_CD.str.contains('I25')].EVENT_ID)
SERO_list=list(pusdig[pusdig.CLIN_CD.str.contains('M05') | pusdig.CLIN_CD.str.contains('M06') ].EVENT_ID)
PSOE_list=list(pusdig[pusdig.CLIN_CD.str.startswith('M07')].EVENT_ID)
LUPU_list=list(pusdig[pusdig.CLIN_CD.str.startswith('M32')].EVENT_ID)
BOWE_list=list(pusdig[pusdig.CLIN_CD.str.startswith('K580')].EVENT_ID)
PSOR_list=list(pusdig[pusdig.CLIN_CD.str.startswith('L40')].EVENT_ID)
ANKY_list=list(pusdig[pusdig.CLIN_CD.str.startswith('M450')].EVENT_ID)
LIVE_list=list(pusdig[pusdig.CLIN_CD.str.startswith('K70') | pusdig.CLIN_CD.str.startswith('K72')   | pusdig.CLIN_CD.str.startswith('K73')  | pusdig.CLIN_CD.str.startswith('K74') ].EVENT_ID)

##add AI Disease Flag
AIDI_list=SERO_list+PSOE_list+LUPU_list+BOWE_list+PSOR_list+ANKY_list

STRK_G1_list=list(pusdig[pusdig.CLIN_CD.isin(['I443','I434'])].EVENT_ID)
STRK_G2_list=list(pusdig[pusdig.CLIN_CD.isin(['I435','G458','G459','G460','G461','G462','G463','G464','G465','G466','G467','G468'])].EVENT_ID)
STRK_G3_list=list(pusdig[pusdig.CLIN_CD.isin(['I436','I437'])].EVENT_ID)
STRK_G4_list=list(pusdig[pusdig.CLIN_CD.str.startswith('I60') 
| pusdig.CLIN_CD.str.startswith('I61') | pusdig.CLIN_CD.str.startswith('I62') | pusdig.CLIN_CD.str.startswith('I63') | pusdig.CLIN_CD.str.startswith('I64')
 | pusdig.CLIN_CD.str.startswith('I65') | pusdig.CLIN_CD.str.startswith('I66') | pusdig.CLIN_CD.str.startswith('I67')].EVENT_ID)
STRK_list=STRK_G1_list+STRK_G2_list+STRK_G3_list+STRK_G4_list

#Acute myocardial infarction
CARD_G1_list=list(pusdig[pusdig.CLIN_CD.isin(['I410','I411','I210','I211','I212','I213','I214','I215','I216','I217','I218','I219','I418'])].EVENT_ID)
#Acute pericarditis
CARD_G2_list=list(pusdig[pusdig.CLIN_CD.isin(['I420','I308','I309'])].EVENT_ID)
#Acute myocarditis
CARD_G3_list=list(pusdig[pusdig.CLIN_CD.isin(['I409','I408','I514','I401','I402','I403','I404','I405','I406','I407'
,'I412','I413','I414','I415','I416','I417','I419','I421','I422'])].EVENT_ID)

CARD_G4_list=list(pusdig[pusdig.CLIN_CD.isin(['I429','I428','I430','I516','I517','I518','I519','I255','I256'])].EVENT_ID)
CARD_G5_list=list(pusdig[pusdig.CLIN_CD.isin(['I500','I501','I509'])].EVENT_ID)
CARD_list=CARD_G1_list+CARD_G2_list+CARD_G3_list+CARD_G4_list+CARD_G5_list

MENI_list=list(pusdig[pusdig.CLIN_CD.isin(['A870','A89','G020','G028','G030','G038','G052','G0331','G039','G04','G05','G934','G312','G92'])].EVENT_ID)
RAMS_list=list(pusdig[pusdig.CLIN_CD.isin(['B022','G510','H490','H491','H492','H493','H494','H510','G511','G519'])].EVENT_ID)

MEDI_G1_list=list(pusdig[pusdig.CLIN_CD.isin(['L0312','L038','L039','M7961','M7962','M7963','M7964','M7968',
                                              'M7969','M7989','T801','T802','T888','I889','I890','I891','L049','R599'])].EVENT_ID)
MEDI_G2_list=list(pusdig[pusdig.CLIN_CD.isin(['T783','T887','T784','L500','L501','L508','L509','T749','T782','T805',
                                              'L0310','L508','R223','R229','L024','L028','L029','L042','L043','L048','L049','L299'])].EVENT_ID)
MEDI_list=MEDI_G1_list+MEDI_G2_list

##add sensitivity disease
WHOC_list=list(pusdig[pusdig.CLIN_CD.isin(['A370','A371','A378','A379'])].EVENT_ID)

APPE_list=list(pusdig[pusdig.CLIN_CD.isin(['K350','K351','K359','K36','K37','K380','K381','K382','K383','K388','K389'])].EVENT_ID)

HERN_list=list(pusdig[pusdig.CLIN_CD.isin(['K4000','K4001','K4010','K4011','K4020','K4021','K4030','K4031','K4040','K4041','K4090','K4091','K410','K411','K412','K413','K414',
'K419','K420','K421','K429','K430','K431','K439','K440','K441','K449','K450','K451','K458','K460','K461','K469'])].EVENT_ID)

DIVE_list=list(pusdig[pusdig.CLIN_CD.isin(['K5700','K5701','K5702','K5703','K5710','K5711','K5712','K513','K5720','K5721','K5722','K5723','K5730','K5731',
'K5732','K5733','K5740','K5741','K5742','K5743','K5780','K5781, K5782, K5783','K5790','K57921','K5792','K5793'])].EVENT_ID)

FEMF_list=list(pusdig[pusdig.CLIN_CD.isin(['S7200','S7201','S7202','S7203','S7204','S7205','S7208','S7210','S7211','S722','S723','S7240','S7241','S7242','S7243','S7244','S727','S728','S729'])].EVENT_ID)

CHOC_list=list(pusdig[pusdig.CLIN_CD.isin(['K8000','K8001','K8010','K8011','K8020','K8021','K8030','K8031','K8040','K8041','K8050','K8051','K8080','K8081','K810',
                                           'K811','K818','K819','K820','K821','K822','K823','K824','K828','K829','K830','K831','K832','K833','K834','K835','K835','K838','K839','K870'])].EVENT_ID)
PAND_list=list(pusdig[pusdig.CLIN_CD.isin(['K85','K860','K861','K862','K863','K868','K869','K871','K903'])].EVENT_ID)
SEPS_list=list(pusdig[pusdig.CLIN_CD.isin(pusdig.CLIN_CD.str.startswith('A40') | pusdig.CLIN_CD.str.startswith('A41'))].EVENT_ID)
HAEM_list=list(pusdig[pusdig.CLIN_CD.isin(['O224','O872']) | pusdig.CLIN_CD.str.startswith('I84')].EVENT_ID)
RENC_list=list(pusdig[pusdig.CLIN_CD.isin(pusdig.CLIN_CD.str.startswith('N20') | pusdig.CLIN_CD.str.startswith('N21') | pusdig.CLIN_CD.str.startswith('N22') | pusdig.CLIN_CD.str.startswith('N23'))].EVENT_ID)
BURN_list=list(pusdig[pusdig.CLIN_CD.isin(pusdig.CLIN_CD.str.startswith('M6137') | pusdig.CLIN_CD.str.startswith('M6138') | pusdig.CLIN_CD.str.startswith('M6139') | pusdig.CLIN_CD.str.startswith('T2')  | pusdig.CLIN_CD.str.startswith('T3'))].EVENT_ID)
LIPO_list=list(pusdig[pusdig.CLIN_CD.isin(pusdig.CLIN_CD.str.startswith('D17'))].EVENT_ID)
EPIS_list=list(pusdig[pusdig.CLIN_CD.isin(['R040'])].EVENT_ID)
CANC_list=list(can.MASTER_ENC.unique())

##Create Level1 Ethnicity group
puseve['ETHNICGP_Lvl1']=puseve['ETHNICGP']//10

puseve["COPD"] = np.where(puseve["EVENT_ID"].isin(COPD_list),1, 0)
puseve["DIAB"] = np.where(puseve["EVENT_ID"].isin(DIAB_list),1, 0)
puseve["GOUT"] = np.where(puseve["EVENT_ID"].isin(GOUT_list),1, 0)
puseve["CERE"] = np.where(puseve["EVENT_ID"].isin(CERE_list),1, 0)
puseve["CHRO"] = np.where(puseve["EVENT_ID"].isin(CHRO_list),1, 0)
puseve["ISCH"] = np.where(puseve["EVENT_ID"].isin(ISCH_list),1, 0)
puseve["SERO"] = np.where(puseve["EVENT_ID"].isin(SERO_list),1, 0)
puseve["PSOE"] = np.where(puseve["EVENT_ID"].isin(PSOE_list),1, 0)
puseve["LUPU"] = np.where(puseve["EVENT_ID"].isin(LUPU_list),1, 0)
puseve["BOWE"] = np.where(puseve["EVENT_ID"].isin(BOWE_list),1, 0)
puseve["PSOR"] = np.where(puseve["EVENT_ID"].isin(PSOR_list),1, 0)
puseve["ANKY"] = np.where(puseve["EVENT_ID"].isin(ANKY_list),1, 0)
puseve["LIVE"] = np.where(puseve["EVENT_ID"].isin(LIVE_list),1, 0)
puseve["AIDI"] = np.where(puseve["EVENT_ID"].isin(AIDI_list),1, 0)
puseve["CANC"] = np.where(puseve["EVENT_ID"].isin(CANC_list),1, 0)

print("Add disease flag for puseve patients")
psize(puseve)

puseve["STRK"] = np.where(puseve["EVENT_ID"].isin(STRK_list),1, 0)
puseve["STRK_G1"] = np.where(puseve["EVENT_ID"].isin(STRK_G1_list),1, 0)
puseve["STRK_G2"] = np.where(puseve["EVENT_ID"].isin(STRK_G2_list),1, 0)
puseve["STRK_G3"] = np.where(puseve["EVENT_ID"].isin(STRK_G3_list),1, 0)
puseve["STRK_G4"] = np.where(puseve["EVENT_ID"].isin(STRK_G4_list),1, 0)

puseve["CARD"] = np.where(puseve["EVENT_ID"].isin(CARD_list),1, 0)
puseve["CARD_G1"] = np.where(puseve["EVENT_ID"].isin(CARD_G1_list),1, 0)
puseve["CARD_G2"] = np.where(puseve["EVENT_ID"].isin(CARD_G2_list),1, 0)
puseve["CARD_G3"] = np.where(puseve["EVENT_ID"].isin(CARD_G3_list),1, 0)
puseve["CARD_G4"] = np.where(puseve["EVENT_ID"].isin(CARD_G4_list),1, 0)
puseve["CARD_G5"] = np.where(puseve["EVENT_ID"].isin(CARD_G5_list),1, 0)

puseve["MENI"] = np.where(puseve["EVENT_ID"].isin(MENI_list),1, 0)
puseve["RAMS"] = np.where(puseve["EVENT_ID"].isin(RAMS_list),1, 0)
puseve["MEDI"] = np.where(puseve["EVENT_ID"].isin(MEDI_list),1, 0)
puseve["MEDI_G1"] = np.where(puseve["EVENT_ID"].isin(MEDI_G1_list),1, 0)
puseve["MEDI_G2"] = np.where(puseve["EVENT_ID"].isin(MEDI_G2_list),1, 0)

puseve["WHOC"] = np.where(puseve["EVENT_ID"].isin(WHOC_list),1, 0)
puseve["APPE"] = np.where(puseve["EVENT_ID"].isin(APPE_list),1, 0)
puseve["HERN"] = np.where(puseve["EVENT_ID"].isin(HERN_list),1, 0)
puseve["DIVE"] = np.where(puseve["EVENT_ID"].isin(DIVE_list),1, 0)
puseve["FEMF"] = np.where(puseve["EVENT_ID"].isin(FEMF_list),1, 0)
puseve["CHOC"] = np.where(puseve["EVENT_ID"].isin(CHOC_list),1, 0)
puseve["PAND"] = np.where(puseve["EVENT_ID"].isin(PAND_list),1, 0)
puseve["SEPS"] = np.where(puseve["EVENT_ID"].isin(SEPS_list),1, 0)
puseve["HAEM"] = np.where(puseve["EVENT_ID"].isin(HAEM_list),1, 0)
puseve["RENC"] = np.where(puseve["EVENT_ID"].isin(RENC_list),1, 0)
puseve["BURN"] = np.where(puseve["EVENT_ID"].isin(BURN_list),1, 0)
puseve["LIPO"] = np.where(puseve["EVENT_ID"].isin(LIPO_list),1, 0)
puseve["EPIS"] = np.where(puseve["EVENT_ID"].isin(EPIS_list),1, 0)


print("Add high risk disease flag for puseve patients")
psize(puseve)

##add DEP Quin
puseve['DEP06_Q']=5
puseve.loc[(puseve.DEP06==2) | (puseve.DEP06==1),"DEP06_Q"]=1
puseve.loc[(puseve.DEP06==4) | (puseve.DEP06==3),"DEP06_Q"]=2
puseve.loc[(puseve.DEP06==6) | (puseve.DEP06==5),"DEP06_Q"]=3
puseve.loc[(puseve.DEP06==8) | (puseve.DEP06==7),"DEP06_Q"]=4
puseve['DEP13_Q']=5
puseve.loc[(puseve.DEP13==2) | (puseve.DEP13==1),"DEP13_Q"]=1
puseve.loc[(puseve.DEP13==4) | (puseve.DEP13==3),"DEP13_Q"]=2
puseve.loc[(puseve.DEP13==6) | (puseve.DEP13==5),"DEP13_Q"]=3
puseve.loc[(puseve.DEP13==8) | (puseve.DEP13==7),"DEP13_Q"]=4

puseve['DEP18_Q']=5
puseve.loc[(puseve.DEP18==2) | (puseve.DEP18==1),"DEP18_Q"]=1
puseve.loc[(puseve.DEP18==4) | (puseve.DEP18==3),"DEP18_Q"]=2
puseve.loc[(puseve.DEP18==6) | (puseve.DEP18==5),"DEP18_Q"]=3
puseve.loc[(puseve.DEP18==8) | (puseve.DEP18==7),"DEP18_Q"]=4

##add DHB Location
puseve.DHBDOM=puseve.DHBDOM.astype(str).str.zfill(3)
puseve=puseve.merge(DHB_Mapping,how="left",on="DHBDOM")

puseve["IMD"] = np.where(puseve["MASTER_ENC"].isin(imd_list),1, 0)

print("Add other category flag for puseve patients")
psize(puseve)


##add cancer type in the puseve dataset
for i in can_list:
  temp_list=list(can[can.MORPH==i].drop_duplicates(keep='first',subset=['MASTER_ENC']).MASTER_ENC)
  puseve[i] = np.where(puseve["MASTER_ENC"].isin(temp_list),1, 0)

print("Add cancer/which type of cancer flag for puseve patients")
psize(puseve)


puseve.loc[:,'EVSTDATE'] = pd.to_datetime(puseve['EVSTDATE'],dayfirst=True)
puseve.loc[:,'VACCINATION_DATE'] = pd.to_datetime(puseve['VACCINATION_DATE'],dayfirst=True)
puseve.loc[:,'DIAG_YEAR']=puseve['EVSTDATE'].dt.year

##Create person days
puseve['Person_Days']=puseve['EVSTDATE']-puseve['VACCINATION_DATE']
puseve['Person_Days']=puseve.Person_Days.dt.days

# puseve.loc[:,'AGE']=pd.cut(puseve['AGE_DIS'], [-1,0,9,19,29,39,49,59,64,69,74,79,150], labels=['1<','1-9','10-19','20-29','30-39','40-49','50-59','60-64','65-69','70-74','75-79','>=80'])
puseve.loc[:,'AGE']=pd.cut(puseve['AGE_DIS'], [-1,54,59,64,69,74,79,150], labels=['50-54','55-59','60-64','65-69','70-74','75-79','>=80'])
puseve_hig_processed=puseve.copy()

control

In [ ]:
pusdig=pusdig_ctr
print('Original pusdig size')
psize(pusdig)
if Type_A_only:
  pusdig=pusdig[pusdig.DIAG_TYP=='A']

print('Type A diagnose pusdig size')
psize(pusdig)

puseve=puseve_ctr.drop_duplicates(['MASTER_ENC','EVENT_ID'])

##Get EVENT_ID for different disease
COPD_list=list(pusdig[pusdig.CLIN_CD.str.contains('J40') | pusdig.CLIN_CD.str.contains('J41') | pusdig.CLIN_CD.str.contains('J42') | pusdig.CLIN_CD.str.contains('J43') | pusdig.CLIN_CD.str.contains('J44')].EVENT_ID)
DIAB_list=list(pusdig[pusdig.CLIN_CD.str.contains('E10') | pusdig.CLIN_CD.str.contains('E11') | pusdig.CLIN_CD.str.contains('E12') | pusdig.CLIN_CD.str.contains('E13') | pusdig.CLIN_CD.str.contains('E14')].EVENT_ID)
GOUT_list=list(pusdig[pusdig.CLIN_CD.str.startswith('M10')].EVENT_ID)
CERE_list=list(pusdig[pusdig.CLIN_CD.str.startswith('I6')].EVENT_ID)
CHRO_list=list(pusdig[pusdig.CLIN_CD.str.startswith('N18')].EVENT_ID)
ISCH_list=list(pusdig[pusdig.CLIN_CD.str.contains('I20') | pusdig.CLIN_CD.str.contains('I21') | pusdig.CLIN_CD.str.contains('I22') | pusdig.CLIN_CD.str.contains('I23') | pusdig.CLIN_CD.str.contains('I24')| pusdig.CLIN_CD.str.contains('I25')].EVENT_ID)
SERO_list=list(pusdig[pusdig.CLIN_CD.str.contains('M05') | pusdig.CLIN_CD.str.contains('M06') ].EVENT_ID)
PSOE_list=list(pusdig[pusdig.CLIN_CD.str.startswith('M07')].EVENT_ID)
LUPU_list=list(pusdig[pusdig.CLIN_CD.str.startswith('M32')].EVENT_ID)
BOWE_list=list(pusdig[pusdig.CLIN_CD.str.startswith('K580')].EVENT_ID)
PSOR_list=list(pusdig[pusdig.CLIN_CD.str.startswith('L40')].EVENT_ID)
ANKY_list=list(pusdig[pusdig.CLIN_CD.str.startswith('M450')].EVENT_ID)
LIVE_list=list(pusdig[pusdig.CLIN_CD.str.startswith('K70') | pusdig.CLIN_CD.str.startswith('K72')   | pusdig.CLIN_CD.str.startswith('K73')  | pusdig.CLIN_CD.str.startswith('K74') ].EVENT_ID)

##add AI Disease Flag
AIDI_list=SERO_list+PSOE_list+LUPU_list+BOWE_list+PSOR_list+ANKY_list

STRK_G1_list=list(pusdig[pusdig.CLIN_CD.isin(['I443','I434'])].EVENT_ID)
STRK_G2_list=list(pusdig[pusdig.CLIN_CD.isin(['I435','G458','G459','G460','G461','G462','G463','G464','G465','G466','G467','G468'])].EVENT_ID)
STRK_G3_list=list(pusdig[pusdig.CLIN_CD.isin(['I436','I437'])].EVENT_ID)
STRK_G4_list=list(pusdig[pusdig.CLIN_CD.str.startswith('I60') 
| pusdig.CLIN_CD.str.startswith('I61') | pusdig.CLIN_CD.str.startswith('I62') | pusdig.CLIN_CD.str.startswith('I63') | pusdig.CLIN_CD.str.startswith('I64')
 | pusdig.CLIN_CD.str.startswith('I65') | pusdig.CLIN_CD.str.startswith('I66') | pusdig.CLIN_CD.str.startswith('I67')].EVENT_ID)
STRK_list=STRK_G1_list+STRK_G2_list+STRK_G3_list+STRK_G4_list

#Acute myocardial infarction
CARD_G1_list=list(pusdig[pusdig.CLIN_CD.isin(['I410','I411','I210','I211','I212','I213','I214','I215','I216','I217','I218','I219','I418'])].EVENT_ID)
#Acute pericarditis
CARD_G2_list=list(pusdig[pusdig.CLIN_CD.isin(['I420','I308','I309'])].EVENT_ID)
#Acute myocarditis
CARD_G3_list=list(pusdig[pusdig.CLIN_CD.isin(['I409','I408','I514','I401','I402','I403','I404','I405','I406','I407'
,'I412','I413','I414','I415','I416','I417','I419','I421','I422'])].EVENT_ID)

CARD_G4_list=list(pusdig[pusdig.CLIN_CD.isin(['I429','I428','I430','I516','I517','I518','I519','I255','I256'])].EVENT_ID)
CARD_G5_list=list(pusdig[pusdig.CLIN_CD.isin(['I500','I501','I509'])].EVENT_ID)
CARD_list=CARD_G1_list+CARD_G2_list+CARD_G3_list+CARD_G4_list+CARD_G5_list

MENI_list=list(pusdig[pusdig.CLIN_CD.isin(['A870','A89','G020','G028','G030','G038','G052','G0331','G039','G04','G05','G934','G312','G92'])].EVENT_ID)
RAMS_list=list(pusdig[pusdig.CLIN_CD.isin(['B022','G510','H490','H491','H492','H493','H494','H510','G511','G519'])].EVENT_ID)

MEDI_G1_list=list(pusdig[pusdig.CLIN_CD.isin(['L0312','L038','L039','M7961','M7962','M7963','M7964','M7968',
                                              'M7969','M7989','T801','T802','T888','I889','I890','I891','L049','R599'])].EVENT_ID)
MEDI_G2_list=list(pusdig[pusdig.CLIN_CD.isin(['T783','T887','T784','L500','L501','L508','L509','T749','T782','T805',
                                              'L0310','L508','R223','R229','L024','L028','L029','L042','L043','L048','L049','L299'])].EVENT_ID)
MEDI_list=MEDI_G1_list+MEDI_G2_list

##add sensitivity disease
WHOC_list=list(pusdig[pusdig.CLIN_CD.isin(['A370','A371','A378','A379'])].EVENT_ID)

APPE_list=list(pusdig[pusdig.CLIN_CD.isin(['K350','K351','K359','K36','K37','K380','K381','K382','K383','K388','K389'])].EVENT_ID)

HERN_list=list(pusdig[pusdig.CLIN_CD.isin(['K4000','K4001','K4010','K4011','K4020','K4021','K4030','K4031','K4040','K4041','K4090','K4091','K410','K411','K412','K413','K414',
'K419','K420','K421','K429','K430','K431','K439','K440','K441','K449','K450','K451','K458','K460','K461','K469'])].EVENT_ID)

DIVE_list=list(pusdig[pusdig.CLIN_CD.isin(['K5700','K5701','K5702','K5703','K5710','K5711','K5712','K513','K5720','K5721','K5722','K5723','K5730','K5731',
'K5732','K5733','K5740','K5741','K5742','K5743','K5780','K5781, K5782, K5783','K5790','K57921','K5792','K5793'])].EVENT_ID)

FEMF_list=list(pusdig[pusdig.CLIN_CD.isin(['S7200','S7201','S7202','S7203','S7204','S7205','S7208','S7210','S7211','S722','S723','S7240','S7241','S7242','S7243','S7244','S727','S728','S729'])].EVENT_ID)

CHOC_list=list(pusdig[pusdig.CLIN_CD.isin(['K8000','K8001','K8010','K8011','K8020','K8021','K8030','K8031','K8040','K8041','K8050','K8051','K8080','K8081','K810',
                                           'K811','K818','K819','K820','K821','K822','K823','K824','K828','K829','K830','K831','K832','K833','K834','K835','K835','K838','K839','K870'])].EVENT_ID)
PAND_list=list(pusdig[pusdig.CLIN_CD.isin(['K85','K860','K861','K862','K863','K868','K869','K871','K903'])].EVENT_ID)
SEPS_list=list(pusdig[pusdig.CLIN_CD.isin(pusdig.CLIN_CD.str.startswith('A40') | pusdig.CLIN_CD.str.startswith('A41'))].EVENT_ID)
HAEM_list=list(pusdig[pusdig.CLIN_CD.isin(['O224','O872']) | pusdig.CLIN_CD.str.startswith('I84')].EVENT_ID)
RENC_list=list(pusdig[pusdig.CLIN_CD.isin(pusdig.CLIN_CD.str.startswith('N20') | pusdig.CLIN_CD.str.startswith('N21') | pusdig.CLIN_CD.str.startswith('N22') | pusdig.CLIN_CD.str.startswith('N23'))].EVENT_ID)
BURN_list=list(pusdig[pusdig.CLIN_CD.isin(pusdig.CLIN_CD.str.startswith('M6137') | pusdig.CLIN_CD.str.startswith('M6138') | pusdig.CLIN_CD.str.startswith('M6139') | pusdig.CLIN_CD.str.startswith('T2')  | pusdig.CLIN_CD.str.startswith('T3'))].EVENT_ID)
LIPO_list=list(pusdig[pusdig.CLIN_CD.isin(pusdig.CLIN_CD.str.startswith('D17'))].EVENT_ID)
EPIS_list=list(pusdig[pusdig.CLIN_CD.isin(['R040'])].EVENT_ID)
CANC_list=list(can.MASTER_ENC.unique())

##Create Level1 Ethnicity group
puseve['ETHNICGP_Lvl1']=puseve['ETHNICGP']//10

puseve["COPD"] = np.where(puseve["EVENT_ID"].isin(COPD_list),1, 0)
puseve["DIAB"] = np.where(puseve["EVENT_ID"].isin(DIAB_list),1, 0)
puseve["GOUT"] = np.where(puseve["EVENT_ID"].isin(GOUT_list),1, 0)
puseve["CERE"] = np.where(puseve["EVENT_ID"].isin(CERE_list),1, 0)
puseve["CHRO"] = np.where(puseve["EVENT_ID"].isin(CHRO_list),1, 0)
puseve["ISCH"] = np.where(puseve["EVENT_ID"].isin(ISCH_list),1, 0)
puseve["SERO"] = np.where(puseve["EVENT_ID"].isin(SERO_list),1, 0)
puseve["PSOE"] = np.where(puseve["EVENT_ID"].isin(PSOE_list),1, 0)
puseve["LUPU"] = np.where(puseve["EVENT_ID"].isin(LUPU_list),1, 0)
puseve["BOWE"] = np.where(puseve["EVENT_ID"].isin(BOWE_list),1, 0)
puseve["PSOR"] = np.where(puseve["EVENT_ID"].isin(PSOR_list),1, 0)
puseve["ANKY"] = np.where(puseve["EVENT_ID"].isin(ANKY_list),1, 0)
puseve["LIVE"] = np.where(puseve["EVENT_ID"].isin(LIVE_list),1, 0)
puseve["AIDI"] = np.where(puseve["EVENT_ID"].isin(AIDI_list),1, 0)
puseve["CANC"] = np.where(puseve["EVENT_ID"].isin(CANC_list),1, 0)

print("Add disease flag for puseve patients")
psize(puseve)

puseve["STRK"] = np.where(puseve["EVENT_ID"].isin(STRK_list),1, 0)
puseve["STRK_G1"] = np.where(puseve["EVENT_ID"].isin(STRK_G1_list),1, 0)
puseve["STRK_G2"] = np.where(puseve["EVENT_ID"].isin(STRK_G2_list),1, 0)
puseve["STRK_G3"] = np.where(puseve["EVENT_ID"].isin(STRK_G3_list),1, 0)
puseve["STRK_G4"] = np.where(puseve["EVENT_ID"].isin(STRK_G4_list),1, 0)

puseve["CARD"] = np.where(puseve["EVENT_ID"].isin(CARD_list),1, 0)
puseve["CARD_G1"] = np.where(puseve["EVENT_ID"].isin(CARD_G1_list),1, 0)
puseve["CARD_G2"] = np.where(puseve["EVENT_ID"].isin(CARD_G2_list),1, 0)
puseve["CARD_G3"] = np.where(puseve["EVENT_ID"].isin(CARD_G3_list),1, 0)
puseve["CARD_G4"] = np.where(puseve["EVENT_ID"].isin(CARD_G4_list),1, 0)
puseve["CARD_G5"] = np.where(puseve["EVENT_ID"].isin(CARD_G5_list),1, 0)

puseve["MENI"] = np.where(puseve["EVENT_ID"].isin(MENI_list),1, 0)
puseve["RAMS"] = np.where(puseve["EVENT_ID"].isin(RAMS_list),1, 0)
puseve["MEDI"] = np.where(puseve["EVENT_ID"].isin(MEDI_list),1, 0)
puseve["MEDI_G1"] = np.where(puseve["EVENT_ID"].isin(MEDI_G1_list),1, 0)
puseve["MEDI_G2"] = np.where(puseve["EVENT_ID"].isin(MEDI_G2_list),1, 0)

puseve["WHOC"] = np.where(puseve["EVENT_ID"].isin(WHOC_list),1, 0)
puseve["APPE"] = np.where(puseve["EVENT_ID"].isin(APPE_list),1, 0)
puseve["HERN"] = np.where(puseve["EVENT_ID"].isin(HERN_list),1, 0)
puseve["DIVE"] = np.where(puseve["EVENT_ID"].isin(DIVE_list),1, 0)
puseve["FEMF"] = np.where(puseve["EVENT_ID"].isin(FEMF_list),1, 0)
puseve["CHOC"] = np.where(puseve["EVENT_ID"].isin(CHOC_list),1, 0)
puseve["PAND"] = np.where(puseve["EVENT_ID"].isin(PAND_list),1, 0)
puseve["SEPS"] = np.where(puseve["EVENT_ID"].isin(SEPS_list),1, 0)
puseve["HAEM"] = np.where(puseve["EVENT_ID"].isin(HAEM_list),1, 0)
puseve["RENC"] = np.where(puseve["EVENT_ID"].isin(RENC_list),1, 0)
puseve["BURN"] = np.where(puseve["EVENT_ID"].isin(BURN_list),1, 0)
puseve["LIPO"] = np.where(puseve["EVENT_ID"].isin(LIPO_list),1, 0)
puseve["EPIS"] = np.where(puseve["EVENT_ID"].isin(EPIS_list),1, 0)


print("Add high risk disease flag for puseve patients")
psize(puseve)

##add DEP Quin
puseve['DEP06_Q']=5
puseve.loc[(puseve.DEP06==2) | (puseve.DEP06==1),"DEP06_Q"]=1
puseve.loc[(puseve.DEP06==4) | (puseve.DEP06==3),"DEP06_Q"]=2
puseve.loc[(puseve.DEP06==6) | (puseve.DEP06==5),"DEP06_Q"]=3
puseve.loc[(puseve.DEP06==8) | (puseve.DEP06==7),"DEP06_Q"]=4
puseve['DEP13_Q']=5
puseve.loc[(puseve.DEP13==2) | (puseve.DEP13==1),"DEP13_Q"]=1
puseve.loc[(puseve.DEP13==4) | (puseve.DEP13==3),"DEP13_Q"]=2
puseve.loc[(puseve.DEP13==6) | (puseve.DEP13==5),"DEP13_Q"]=3
puseve.loc[(puseve.DEP13==8) | (puseve.DEP13==7),"DEP13_Q"]=4

puseve['DEP18_Q']=5
puseve.loc[(puseve.DEP18==2) | (puseve.DEP18==1),"DEP18_Q"]=1
puseve.loc[(puseve.DEP18==4) | (puseve.DEP18==3),"DEP18_Q"]=2
puseve.loc[(puseve.DEP18==6) | (puseve.DEP18==5),"DEP18_Q"]=3
puseve.loc[(puseve.DEP18==8) | (puseve.DEP18==7),"DEP18_Q"]=4

##add DHB Location
puseve.DHBDOM=puseve.DHBDOM.astype(str).str.zfill(3)
puseve=puseve.merge(DHB_Mapping,how="left",on="DHBDOM")

puseve["IMD"] = np.where(puseve["MASTER_ENC"].isin(imd_list),1, 0)

print("Add other category flag for puseve patients")
psize(puseve)


##add cancer type in the puseve dataset
for i in can_list:
  temp_list=list(can[can.MORPH==i].drop_duplicates(keep='first',subset=['MASTER_ENC']).MASTER_ENC)
  puseve[i] = np.where(puseve["MASTER_ENC"].isin(temp_list),1, 0)

print("Add cancer/which type of cancer flag for puseve patients")
psize(puseve)


puseve.loc[:,'EVSTDATE'] = pd.to_datetime(puseve['EVSTDATE'],dayfirst=True)
puseve.loc[:,'VACCINATION_DATE'] = pd.to_datetime(puseve['VACCINATION_DATE'],dayfirst=True)
puseve.loc[:,'DIAG_YEAR']=puseve['EVSTDATE'].dt.year

##Create person days
puseve['Person_Days']=puseve['EVSTDATE']-puseve['VACCINATION_DATE']
puseve['Person_Days']=puseve.Person_Days.dt.days

# puseve.loc[:,'AGE']=pd.cut(puseve['AGE_DIS'], [-1,0,9,19,29,39,49,59,64,69,74,79,150], labels=['1<','1-9','10-19','20-29','30-39','40-49','50-59','60-64','65-69','70-74','75-79','>=80'])
puseve.loc[:,'AGE']=pd.cut(puseve['AGE_DIS'], [-1,54,59,64,69,74,79,150], labels=['50-54','55-59','60-64','65-69','70-74','75-79','>=80'])
puseve_ctr_processed=puseve.copy()

Original pusdig size
(18467, 5)
Type A diagnose pusdig size
(1868, 5)
Add disease flag for puseve patients
(1868, 48)
Add high risk disease flag for puseve patients
(1868, 77)
Add other category flag for puseve patients
(1868, 82)
Add cancer/which type of cancer flag for puseve patients
(1868, 112)


##Table1

In [ ]:
puseve_hig_processed['FLAG']=1
puseve_ctr_processed['FLAG']=0
puseve_full=pd.concat([puseve_hig_processed,puseve_ctr_processed],axis=0)

data=puseve_full.copy()

In [ ]:
data.MASTER_ENC.nunique()

In [ ]:
pyramid=data.groupby(['AGE','GENDER'])['AGE_DIS'].agg(['count']).reset_index()
pyramid['count']=np.where(pyramid.GENDER=='M',pyramid['count']*-1,pyramid['count'])

df=pyramid.copy()
# Draw Plot
plt.figure(figsize=(10,6), dpi= 80)
group_col = 'GENDER'
order_of_bars = df.AGE.unique()[::-1]
colors = [plt.cm.Spectral(i/float(len(df[group_col].unique())-1)) for i in range(len(df[group_col].unique()))]

for c, group in zip(colors, df[group_col].unique()):
    sns.barplot(x='count', y='AGE', data=df.loc[df[group_col]==group, :], order=order_of_bars, color=c, label=group)

# Decorations
plt.xlabel("$cases$")
plt.ylabel("AGE of cases")
xticks = plt.gca().get_xticks().astype(np.int)
plt.xticks(xticks, labels=np.abs(xticks));
plt.yticks(fontsize=12)
plt.title("Population Pyramid of the cases", fontsize=22)
plt.legend()
plt.savefig('pyramid.pdf',dpi=500)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


#Table for events level

In [ ]:
for i in table_list+disease_list:
  display(data.pivot_table(index=[i],columns='FLAG',values='MASTER_ENC',aggfunc=np.count_nonzero,margins=True))
  display(data.pivot_table(index=[i],columns='FLAG',values='MASTER_ENC',aggfunc=np.count_nonzero,margins=True).transform(lambda x: x*200/x.sum()))

FLAG,0,1,All
DIAG_YEAR,,,
2018,1152,468,1620
2019,418,138,556
2020,191,53,244
2021,107,26,133
All,1868,685,2553


FLAG,0,1,All
DIAG_YEAR,,,
2018,61.670236,68.321168,63.454759
2019,22.376874,20.145985,21.778300
2020,10.224839,7.737226,9.557383
2021,5.728051,3.795620,5.209557
All,100.000000,100.000000,100.000000


FLAG,0,1,All
AGE,,,
50-54,NaN,1.000000,1
55-59,1.000000,1.000000,2
60-64,3.000000,5.000000,8
65-69,422.000000,163.000000,585
70-74,560.000000,214.000000,774
75-79,671.000000,238.000000,909
>=80,211.000000,63.000000,274
All,1868.000000,685.000000,2553


FLAG,0,1,All
AGE,,,
50-54,NaN,0.145985,0.039170
55-59,0.053533,0.145985,0.078339
60-64,0.160600,0.729927,0.313357
65-69,22.591006,23.795620,22.914219
70-74,29.978587,31.240876,30.317274
75-79,35.920771,34.744526,35.605170
>=80,11.295503,9.197080,10.732472
All,100.000000,100.000000,100.000000


FLAG,0,1,All
GENDER,,,
F,790,312,1102
M,1078,373,1451
All,1868,685,2553


...

FLAG,0,1,All
BURN,,,
0,100.000000,100.000000,100.000000
All,100.000000,100.000000,100.000000


FLAG,0,1,All
LIPO,,,
0,1868,685,2553
All,1868,685,2553


FLAG,0,1,All
LIPO,,,
0,100.000000,100.000000,100.000000
All,100.000000,100.000000,100.000000


FLAG,0,1,All
EPIS,,,
0,1856,681,2537
1,12,4,16
All,1868,685,2553


FLAG,0,1,All
EPIS,,,
0,99.357602,99.416058,99.373286
1,0.642398,0.583942,0.626714
All,100.000000,100.000000,100.000000


#Table for individual level

In [ ]:
data.shape

In [ ]:
data1a=data.copy()
data1a['ADVERSE_FLAG']=data1a['STRK']+data1a['CARD']+data1a['MENI']+data1a['RAMS']+data1a['MEDI']
data1a=data1a[data1a.ADVERSE_FLAG>0]
data1a.shape

In [ ]:
data1a=data.groupby(['MASTER_ENC']).max().reset_index()

In [ ]:
for i in table_list+disease_list:
  display(data1a.pivot_table(index=[i],columns='FLAG',values='MASTER_ENC',aggfunc=np.count_nonzero,margins=True))
  display(data1a.pivot_table(index=[i],columns='FLAG',values='MASTER_ENC',aggfunc=np.count_nonzero,margins=True).transform(lambda x: x*200/x.sum()))

###Descriptive statistics - mean, std etc.


In [ ]:
data['AGE_DIS'].describe()

In [ ]:
for i in table_list+disease_list:
  display(data.groupby([i])['AGE_DIS'].agg(['mean', 'std']))

In [ ]:
for i in table_list+disease_list:
  display(data.groupby([i,'FLAG'])['AGE_DIS'].agg(['mean', 'std']))

#Analyze - Type A only for side effect

In [ ]:
##Return Primary Diagnose Only?
Type_A_only=True

**High RISK GROUP 

In [ ]:
pusdig=pusdig_hig
print('Original pusdig size')
psize(pusdig)
if Type_A_only:
  pusdig=pusdig[pusdig.DIAG_TYP=='A']

print('Type A diagnose pusdig size')
psize(pusdig)

puseve=puseve_hig.drop_duplicates(['MASTER_ENC','EVENT_ID'])

##Get EVENT_ID for different disease
COPD_list=list(pusdig[pusdig.CLIN_CD.str.contains('J40') | pusdig.CLIN_CD.str.contains('J41') | pusdig.CLIN_CD.str.contains('J42') | pusdig.CLIN_CD.str.contains('J43') | pusdig.CLIN_CD.str.contains('J44')].EVENT_ID)
DIAB_list=list(pusdig[pusdig.CLIN_CD.str.contains('E10') | pusdig.CLIN_CD.str.contains('E11') | pusdig.CLIN_CD.str.contains('E12') | pusdig.CLIN_CD.str.contains('E13') | pusdig.CLIN_CD.str.contains('E14')].EVENT_ID)
GOUT_list=list(pusdig[pusdig.CLIN_CD.str.startswith('M10')].EVENT_ID)
CERE_list=list(pusdig[pusdig.CLIN_CD.str.startswith('I6')].EVENT_ID)
CHRO_list=list(pusdig[pusdig.CLIN_CD.str.startswith('N18')].EVENT_ID)
ISCH_list=list(pusdig[pusdig.CLIN_CD.str.contains('I20') | pusdig.CLIN_CD.str.contains('I21') | pusdig.CLIN_CD.str.contains('I22') | pusdig.CLIN_CD.str.contains('I23') | pusdig.CLIN_CD.str.contains('I24')| pusdig.CLIN_CD.str.contains('I25')].EVENT_ID)
SERO_list=list(pusdig[pusdig.CLIN_CD.str.contains('M05') | pusdig.CLIN_CD.str.contains('M06') ].EVENT_ID)
PSOE_list=list(pusdig[pusdig.CLIN_CD.str.startswith('M07')].EVENT_ID)
LUPU_list=list(pusdig[pusdig.CLIN_CD.str.startswith('M32')].EVENT_ID)
BOWE_list=list(pusdig[pusdig.CLIN_CD.str.startswith('K580')].EVENT_ID)
PSOR_list=list(pusdig[pusdig.CLIN_CD.str.startswith('L40')].EVENT_ID)
ANKY_list=list(pusdig[pusdig.CLIN_CD.str.startswith('M450')].EVENT_ID)
LIVE_list=list(pusdig[pusdig.CLIN_CD.str.startswith('K70') | pusdig.CLIN_CD.str.startswith('K72')   | pusdig.CLIN_CD.str.startswith('K73')  | pusdig.CLIN_CD.str.startswith('K74') ].EVENT_ID)

##add AI Disease Flag
AIDI_list=SERO_list+PSOE_list+LUPU_list+BOWE_list+PSOR_list+ANKY_list

STRK_G1_list=list(pusdig[pusdig.CLIN_CD.isin(['I443','I434'])].EVENT_ID)
STRK_G2_list=list(pusdig[pusdig.CLIN_CD.isin(['I435','G458','G459','G460','G461','G462','G463','G464','G465','G466','G467','G468'])].EVENT_ID)
STRK_G3_list=list(pusdig[pusdig.CLIN_CD.isin(['I436','I437'])].EVENT_ID)
STRK_G4_list=list(pusdig[pusdig.CLIN_CD.str.startswith('I60') 
| pusdig.CLIN_CD.str.startswith('I61') | pusdig.CLIN_CD.str.startswith('I62') | pusdig.CLIN_CD.str.startswith('I63') | pusdig.CLIN_CD.str.startswith('I64')
 | pusdig.CLIN_CD.str.startswith('I65') | pusdig.CLIN_CD.str.startswith('I66') | pusdig.CLIN_CD.str.startswith('I67')].EVENT_ID)
STRK_list=STRK_G1_list+STRK_G2_list+STRK_G3_list+STRK_G4_list

#Acute myocardial infarction
CARD_G1_list=list(pusdig[pusdig.CLIN_CD.isin(['I410','I411','I210','I211','I212','I213','I214','I215','I216','I217','I218','I219','I418'])].EVENT_ID)
#Acute pericarditis
CARD_G2_list=list(pusdig[pusdig.CLIN_CD.isin(['I420','I308','I309'])].EVENT_ID)
#Acute myocarditis
CARD_G3_list=list(pusdig[pusdig.CLIN_CD.isin(['I409','I408','I514','I401','I402','I403','I404','I405','I406','I407'
,'I412','I413','I414','I415','I416','I417','I419','I421','I422'])].EVENT_ID)

CARD_G4_list=list(pusdig[pusdig.CLIN_CD.isin(['I429','I428','I430','I516','I517','I518','I519','I255','I256'])].EVENT_ID)
CARD_G5_list=list(pusdig[pusdig.CLIN_CD.isin(['I500','I501','I509'])].EVENT_ID)
CARD_list=CARD_G1_list+CARD_G2_list+CARD_G3_list+CARD_G4_list+CARD_G5_list

MENI_list=list(pusdig[pusdig.CLIN_CD.isin(['A870','A89','G020','G028','G030','G038','G052','G0331','G039','G04','G05','G934','G312','G92'])].EVENT_ID)
RAMS_list=list(pusdig[pusdig.CLIN_CD.isin(['B022','G510','H490','H491','H492','H493','H494','H510','G511','G519'])].EVENT_ID)

MEDI_G1_list=list(pusdig[pusdig.CLIN_CD.isin(['L0312','L038','L039','M7961','M7962','M7963','M7964','M7968',
                                              'M7969','M7989','T801','T802','T888','I889','I890','I891','L049','R599'])].EVENT_ID)
MEDI_G2_list=list(pusdig[pusdig.CLIN_CD.isin(['T783','T887','T784','L500','L501','L508','L509','T749','T782','T805',
                                              'L0310','L508','R223','R229','L024','L028','L029','L042','L043','L048','L049','L299'])].EVENT_ID)
MEDI_list=MEDI_G1_list+MEDI_G2_list

##add sensitivity disease
WHOC_list=list(pusdig[pusdig.CLIN_CD.isin(['A370','A371','A378','A379'])].EVENT_ID)

APPE_list=list(pusdig[pusdig.CLIN_CD.isin(['K350','K351','K359','K36','K37','K380','K381','K382','K383','K388','K389'])].EVENT_ID)

HERN_list=list(pusdig[pusdig.CLIN_CD.isin(['K4000','K4001','K4010','K4011','K4020','K4021','K4030','K4031','K4040','K4041','K4090','K4091','K410','K411','K412','K413','K414',
'K419','K420','K421','K429','K430','K431','K439','K440','K441','K449','K450','K451','K458','K460','K461','K469'])].EVENT_ID)

DIVE_list=list(pusdig[pusdig.CLIN_CD.isin(['K5700','K5701','K5702','K5703','K5710','K5711','K5712','K513','K5720','K5721','K5722','K5723','K5730','K5731',
'K5732','K5733','K5740','K5741','K5742','K5743','K5780','K5781, K5782, K5783','K5790','K57921','K5792','K5793'])].EVENT_ID)

FEMF_list=list(pusdig[pusdig.CLIN_CD.isin(['S7200','S7201','S7202','S7203','S7204','S7205','S7208','S7210','S7211','S722','S723','S7240','S7241','S7242','S7243','S7244','S727','S728','S729'])].EVENT_ID)

CHOC_list=list(pusdig[pusdig.CLIN_CD.isin(['K8000','K8001','K8010','K8011','K8020','K8021','K8030','K8031','K8040','K8041','K8050','K8051','K8080','K8081','K810',
                                           'K811','K818','K819','K820','K821','K822','K823','K824','K828','K829','K830','K831','K832','K833','K834','K835','K835','K838','K839','K870'])].EVENT_ID)
PAND_list=list(pusdig[pusdig.CLIN_CD.isin(['K85','K860','K861','K862','K863','K868','K869','K871','K903'])].EVENT_ID)
SEPS_list=list(pusdig[pusdig.CLIN_CD.isin(pusdig.CLIN_CD.str.startswith('A40') | pusdig.CLIN_CD.str.startswith('A41'))].EVENT_ID)
HAEM_list=list(pusdig[pusdig.CLIN_CD.isin(['O224','O872']) | pusdig.CLIN_CD.str.startswith('I84')].EVENT_ID)
RENC_list=list(pusdig[pusdig.CLIN_CD.isin(pusdig.CLIN_CD.str.startswith('N20') | pusdig.CLIN_CD.str.startswith('N21') | pusdig.CLIN_CD.str.startswith('N22') | pusdig.CLIN_CD.str.startswith('N23'))].EVENT_ID)
BURN_list=list(pusdig[pusdig.CLIN_CD.isin(pusdig.CLIN_CD.str.startswith('M6137') | pusdig.CLIN_CD.str.startswith('M6138') | pusdig.CLIN_CD.str.startswith('M6139') | pusdig.CLIN_CD.str.startswith('T2')  | pusdig.CLIN_CD.str.startswith('T3'))].EVENT_ID)
LIPO_list=list(pusdig[pusdig.CLIN_CD.isin(pusdig.CLIN_CD.str.startswith('D17'))].EVENT_ID)
EPIS_list=list(pusdig[pusdig.CLIN_CD.isin(['R040'])].EVENT_ID)
CANC_list=list(can.MASTER_ENC.unique())

##Create Level1 Ethnicity group
puseve['ETHNICGP_Lvl1']=puseve['ETHNICGP']//10

puseve["COPD"] = np.where(puseve["EVENT_ID"].isin(COPD_list),1, 0)
puseve["DIAB"] = np.where(puseve["EVENT_ID"].isin(DIAB_list),1, 0)
puseve["GOUT"] = np.where(puseve["EVENT_ID"].isin(GOUT_list),1, 0)
puseve["CERE"] = np.where(puseve["EVENT_ID"].isin(CERE_list),1, 0)
puseve["CHRO"] = np.where(puseve["EVENT_ID"].isin(CHRO_list),1, 0)
puseve["ISCH"] = np.where(puseve["EVENT_ID"].isin(ISCH_list),1, 0)
puseve["SERO"] = np.where(puseve["EVENT_ID"].isin(SERO_list),1, 0)
puseve["PSOE"] = np.where(puseve["EVENT_ID"].isin(PSOE_list),1, 0)
puseve["LUPU"] = np.where(puseve["EVENT_ID"].isin(LUPU_list),1, 0)
puseve["BOWE"] = np.where(puseve["EVENT_ID"].isin(BOWE_list),1, 0)
puseve["PSOR"] = np.where(puseve["EVENT_ID"].isin(PSOR_list),1, 0)
puseve["ANKY"] = np.where(puseve["EVENT_ID"].isin(ANKY_list),1, 0)
puseve["LIVE"] = np.where(puseve["EVENT_ID"].isin(LIVE_list),1, 0)
puseve["AIDI"] = np.where(puseve["EVENT_ID"].isin(AIDI_list),1, 0)
puseve["CANC"] = np.where(puseve["EVENT_ID"].isin(CANC_list),1, 0)

print("Add disease flag for puseve patients")
psize(puseve)

puseve["STRK"] = np.where(puseve["EVENT_ID"].isin(STRK_list),1, 0)
puseve["STRK_G1"] = np.where(puseve["EVENT_ID"].isin(STRK_G1_list),1, 0)
puseve["STRK_G2"] = np.where(puseve["EVENT_ID"].isin(STRK_G2_list),1, 0)
puseve["STRK_G3"] = np.where(puseve["EVENT_ID"].isin(STRK_G3_list),1, 0)
puseve["STRK_G4"] = np.where(puseve["EVENT_ID"].isin(STRK_G4_list),1, 0)

puseve["CARD"] = np.where(puseve["EVENT_ID"].isin(CARD_list),1, 0)
puseve["CARD_G1"] = np.where(puseve["EVENT_ID"].isin(CARD_G1_list),1, 0)
puseve["CARD_G2"] = np.where(puseve["EVENT_ID"].isin(CARD_G2_list),1, 0)
puseve["CARD_G3"] = np.where(puseve["EVENT_ID"].isin(CARD_G3_list),1, 0)
puseve["CARD_G4"] = np.where(puseve["EVENT_ID"].isin(CARD_G4_list),1, 0)
puseve["CARD_G5"] = np.where(puseve["EVENT_ID"].isin(CARD_G5_list),1, 0)

puseve["MENI"] = np.where(puseve["EVENT_ID"].isin(MENI_list),1, 0)
puseve["RAMS"] = np.where(puseve["EVENT_ID"].isin(RAMS_list),1, 0)
puseve["MEDI"] = np.where(puseve["EVENT_ID"].isin(MEDI_list),1, 0)
puseve["MEDI_G1"] = np.where(puseve["EVENT_ID"].isin(MEDI_G1_list),1, 0)
puseve["MEDI_G2"] = np.where(puseve["EVENT_ID"].isin(MEDI_G2_list),1, 0)

puseve["WHOC"] = np.where(puseve["EVENT_ID"].isin(WHOC_list),1, 0)
puseve["APPE"] = np.where(puseve["EVENT_ID"].isin(APPE_list),1, 0)
puseve["HERN"] = np.where(puseve["EVENT_ID"].isin(HERN_list),1, 0)
puseve["DIVE"] = np.where(puseve["EVENT_ID"].isin(DIVE_list),1, 0)
puseve["FEMF"] = np.where(puseve["EVENT_ID"].isin(FEMF_list),1, 0)
puseve["CHOC"] = np.where(puseve["EVENT_ID"].isin(CHOC_list),1, 0)
puseve["PAND"] = np.where(puseve["EVENT_ID"].isin(PAND_list),1, 0)
puseve["SEPS"] = np.where(puseve["EVENT_ID"].isin(SEPS_list),1, 0)
puseve["HAEM"] = np.where(puseve["EVENT_ID"].isin(HAEM_list),1, 0)
puseve["RENC"] = np.where(puseve["EVENT_ID"].isin(RENC_list),1, 0)
puseve["BURN"] = np.where(puseve["EVENT_ID"].isin(BURN_list),1, 0)
puseve["LIPO"] = np.where(puseve["EVENT_ID"].isin(LIPO_list),1, 0)
puseve["EPIS"] = np.where(puseve["EVENT_ID"].isin(EPIS_list),1, 0)


print("Add high risk disease flag for puseve patients")
psize(puseve)

##add DEP Quin
puseve['DEP06_Q']=5
puseve.loc[(puseve.DEP06==2) | (puseve.DEP06==1),"DEP06_Q"]=1
puseve.loc[(puseve.DEP06==4) | (puseve.DEP06==3),"DEP06_Q"]=2
puseve.loc[(puseve.DEP06==6) | (puseve.DEP06==5),"DEP06_Q"]=3
puseve.loc[(puseve.DEP06==8) | (puseve.DEP06==7),"DEP06_Q"]=4
puseve['DEP13_Q']=5
puseve.loc[(puseve.DEP13==2) | (puseve.DEP13==1),"DEP13_Q"]=1
puseve.loc[(puseve.DEP13==4) | (puseve.DEP13==3),"DEP13_Q"]=2
puseve.loc[(puseve.DEP13==6) | (puseve.DEP13==5),"DEP13_Q"]=3
puseve.loc[(puseve.DEP13==8) | (puseve.DEP13==7),"DEP13_Q"]=4

puseve['DEP18_Q']=5
puseve.loc[(puseve.DEP18==2) | (puseve.DEP18==1),"DEP18_Q"]=1
puseve.loc[(puseve.DEP18==4) | (puseve.DEP18==3),"DEP18_Q"]=2
puseve.loc[(puseve.DEP18==6) | (puseve.DEP18==5),"DEP18_Q"]=3
puseve.loc[(puseve.DEP18==8) | (puseve.DEP18==7),"DEP18_Q"]=4

##add DHB Location
puseve.DHBDOM=puseve.DHBDOM.astype(str).str.zfill(3)
puseve=puseve.merge(DHB_Mapping,how="left",on="DHBDOM")

puseve["IMD"] = np.where(puseve["MASTER_ENC"].isin(imd_list),1, 0)

print("Add other category flag for puseve patients")
psize(puseve)


##add cancer type in the puseve dataset
for i in can_list:
  temp_list=list(can[can.MORPH==i].drop_duplicates(keep='first',subset=['MASTER_ENC']).MASTER_ENC)
  puseve[i] = np.where(puseve["MASTER_ENC"].isin(temp_list),1, 0)

print("Add cancer/which type of cancer flag for puseve patients")
psize(puseve)


puseve.loc[:,'EVSTDATE'] = pd.to_datetime(puseve['EVSTDATE'],dayfirst=True)
puseve.loc[:,'EVSTDATE'] = pd.to_datetime(puseve['EVSTDATE'],dayfirst=True)

puseve.loc[:,'VACCINATION_DATE'] = pd.to_datetime(puseve['VACCINATION_DATE'],dayfirst=True)
puseve.loc[:,'DIAG_YEAR']=puseve['EVSTDATE'].dt.year

##Create person days
puseve['Person_Days']=puseve['EVSTDATE']-puseve['VACCINATION_DATE']
puseve['Person_Days']=puseve.Person_Days.dt.days

# puseve.loc[:,'AGE']=pd.cut(puseve['AGE_DIS'], [-1,0,9,19,29,39,49,59,64,69,74,79,150], labels=['1<','1-9','10-19','20-29','30-39','40-49','50-59','60-64','65-69','70-74','75-79','>=80'])
puseve.loc[:,'AGE']=pd.cut(puseve['AGE_DIS'], [-1,54,59,64,69,74,79,150], labels=['50-54','55-59','60-64','65-69','70-74','75-79','>=80'])
puseve_hig_processed=puseve.copy()

Lookback

In [ ]:
pusdig=pusdig_higlb
print('Original pusdig size')
psize(pusdig)
if Type_A_only:
  pusdig=pusdig[pusdig.DIAG_TYP=='A']

print('Type A diagnose pusdig size')
psize(pusdig)

puseve=puseve_higlb.drop_duplicates(['MASTER_ENC','EVENT_ID'])

##add sensitivity disease
WHOC_list=list(pusdig[pusdig.CLIN_CD.isin(['A370','A371','A378','A379'])].EVENT_ID)

APPE_list=list(pusdig[pusdig.CLIN_CD.isin(['K350','K351','K359','K36','K37','K380','K381','K382','K383','K388','K389'])].EVENT_ID)

HERN_list=list(pusdig[pusdig.CLIN_CD.isin(['K4000','K4001','K4010','K4011','K4020','K4021','K4030','K4031','K4040','K4041','K4090','K4091','K410','K411','K412','K413','K414',
'K419','K420','K421','K429','K430','K431','K439','K440','K441','K449','K450','K451','K458','K460','K461','K469'])].EVENT_ID)

DIVE_list=list(pusdig[pusdig.CLIN_CD.isin(['K5700','K5701','K5702','K5703','K5710','K5711','K5712','K513','K5720','K5721','K5722','K5723','K5730','K5731',
'K5732','K5733','K5740','K5741','K5742','K5743','K5780','K5781, K5782, K5783','K5790','K57921','K5792','K5793'])].EVENT_ID)

FEMF_list=list(pusdig[pusdig.CLIN_CD.isin(['S7200','S7201','S7202','S7203','S7204','S7205','S7208','S7210','S7211','S722','S723','S7240','S7241','S7242','S7243','S7244','S727','S728','S729'])].EVENT_ID)

CHOC_list=list(pusdig[pusdig.CLIN_CD.isin(['K8000','K8001','K8010','K8011','K8020','K8021','K8030','K8031','K8040','K8041','K8050','K8051','K8080','K8081','K810',
                                           'K811','K818','K819','K820','K821','K822','K823','K824','K828','K829','K830','K831','K832','K833','K834','K835','K835','K838','K839','K870'])].EVENT_ID)
PAND_list=list(pusdig[pusdig.CLIN_CD.isin(['K85','K860','K861','K862','K863','K868','K869','K871','K903'])].EVENT_ID)
SEPS_list=list(pusdig[pusdig.CLIN_CD.isin(pusdig.CLIN_CD.str.startswith('A40') | pusdig.CLIN_CD.str.startswith('A41'))].EVENT_ID)
HAEM_list=list(pusdig[pusdig.CLIN_CD.isin(['O224','O872']) | pusdig.CLIN_CD.str.startswith('I84')].EVENT_ID)
RENC_list=list(pusdig[pusdig.CLIN_CD.isin(pusdig.CLIN_CD.str.startswith('N20') | pusdig.CLIN_CD.str.startswith('N21') | pusdig.CLIN_CD.str.startswith('N22') | pusdig.CLIN_CD.str.startswith('N23'))].EVENT_ID)
BURN_list=list(pusdig[pusdig.CLIN_CD.isin(pusdig.CLIN_CD.str.startswith('M6137') | pusdig.CLIN_CD.str.startswith('M6138') | pusdig.CLIN_CD.str.startswith('M6139') | pusdig.CLIN_CD.str.startswith('T2')  | pusdig.CLIN_CD.str.startswith('T3'))].EVENT_ID)
LIPO_list=list(pusdig[pusdig.CLIN_CD.isin(pusdig.CLIN_CD.str.startswith('D17'))].EVENT_ID)
EPIS_list=list(pusdig[pusdig.CLIN_CD.isin(['R040'])].EVENT_ID)
CANC_list=list(can.MASTER_ENC.unique())

puseve["WHOC"] = np.where(puseve["EVENT_ID"].isin(WHOC_list),1, 0)
puseve["APPE"] = np.where(puseve["EVENT_ID"].isin(APPE_list),1, 0)
puseve["HERN"] = np.where(puseve["EVENT_ID"].isin(HERN_list),1, 0)
puseve["DIVE"] = np.where(puseve["EVENT_ID"].isin(DIVE_list),1, 0)
puseve["FEMF"] = np.where(puseve["EVENT_ID"].isin(FEMF_list),1, 0)
puseve["CHOC"] = np.where(puseve["EVENT_ID"].isin(CHOC_list),1, 0)
puseve["PAND"] = np.where(puseve["EVENT_ID"].isin(PAND_list),1, 0)
puseve["SEPS"] = np.where(puseve["EVENT_ID"].isin(SEPS_list),1, 0)
puseve["HAEM"] = np.where(puseve["EVENT_ID"].isin(HAEM_list),1, 0)
puseve["RENC"] = np.where(puseve["EVENT_ID"].isin(RENC_list),1, 0)
puseve["BURN"] = np.where(puseve["EVENT_ID"].isin(BURN_list),1, 0)
puseve["LIPO"] = np.where(puseve["EVENT_ID"].isin(LIPO_list),1, 0)
puseve["EPIS"] = np.where(puseve["EVENT_ID"].isin(EPIS_list),1, 0)


print("Add high risk disease flag for puseve patients")
psize(puseve)

puseve_higlb_processed=puseve.copy()

**Control GROUP

In [ ]:
pusdig=pusdig_ctr
print('Original pusdig size')
psize(pusdig)
if Type_A_only:
  pusdig=pusdig[pusdig.DIAG_TYP=='A']

print('Type A diagnose pusdig size')
psize(pusdig)

puseve=puseve_ctr.drop_duplicates(['MASTER_ENC','EVENT_ID'])

##Get EVENT_ID for different disease
COPD_list=list(pusdig[pusdig.CLIN_CD.str.contains('J40') | pusdig.CLIN_CD.str.contains('J41') | pusdig.CLIN_CD.str.contains('J42') | pusdig.CLIN_CD.str.contains('J43') | pusdig.CLIN_CD.str.contains('J44')].EVENT_ID)
DIAB_list=list(pusdig[pusdig.CLIN_CD.str.contains('E10') | pusdig.CLIN_CD.str.contains('E11') | pusdig.CLIN_CD.str.contains('E12') | pusdig.CLIN_CD.str.contains('E13') | pusdig.CLIN_CD.str.contains('E14')].EVENT_ID)
GOUT_list=list(pusdig[pusdig.CLIN_CD.str.startswith('M10')].EVENT_ID)
CERE_list=list(pusdig[pusdig.CLIN_CD.str.startswith('I6')].EVENT_ID)
CHRO_list=list(pusdig[pusdig.CLIN_CD.str.startswith('N18')].EVENT_ID)
ISCH_list=list(pusdig[pusdig.CLIN_CD.str.contains('I20') | pusdig.CLIN_CD.str.contains('I21') | pusdig.CLIN_CD.str.contains('I22') | pusdig.CLIN_CD.str.contains('I23') | pusdig.CLIN_CD.str.contains('I24')| pusdig.CLIN_CD.str.contains('I25')].EVENT_ID)
SERO_list=list(pusdig[pusdig.CLIN_CD.str.contains('M05') | pusdig.CLIN_CD.str.contains('M06') ].EVENT_ID)
PSOE_list=list(pusdig[pusdig.CLIN_CD.str.startswith('M07')].EVENT_ID)
LUPU_list=list(pusdig[pusdig.CLIN_CD.str.startswith('M32')].EVENT_ID)
BOWE_list=list(pusdig[pusdig.CLIN_CD.str.startswith('K580')].EVENT_ID)
PSOR_list=list(pusdig[pusdig.CLIN_CD.str.startswith('L40')].EVENT_ID)
ANKY_list=list(pusdig[pusdig.CLIN_CD.str.startswith('M450')].EVENT_ID)
LIVE_list=list(pusdig[pusdig.CLIN_CD.str.startswith('K70') | pusdig.CLIN_CD.str.startswith('K72')   | pusdig.CLIN_CD.str.startswith('K73')  | pusdig.CLIN_CD.str.startswith('K74') ].EVENT_ID)

##add AI Disease Flag
AIDI_list=SERO_list+PSOE_list+LUPU_list+BOWE_list+PSOR_list+ANKY_list

STRK_G1_list=list(pusdig[pusdig.CLIN_CD.isin(['I443','I434'])].EVENT_ID)
STRK_G2_list=list(pusdig[pusdig.CLIN_CD.isin(['I435','G458','G459','G460','G461','G462','G463','G464','G465','G466','G467','G468'])].EVENT_ID)
STRK_G3_list=list(pusdig[pusdig.CLIN_CD.isin(['I436','I437'])].EVENT_ID)
STRK_G4_list=list(pusdig[pusdig.CLIN_CD.str.startswith('I60') 
| pusdig.CLIN_CD.str.startswith('I61') | pusdig.CLIN_CD.str.startswith('I62') | pusdig.CLIN_CD.str.startswith('I63') | pusdig.CLIN_CD.str.startswith('I64')
 | pusdig.CLIN_CD.str.startswith('I65') | pusdig.CLIN_CD.str.startswith('I66') | pusdig.CLIN_CD.str.startswith('I67')].EVENT_ID)
STRK_list=STRK_G1_list+STRK_G2_list+STRK_G3_list+STRK_G4_list

#Acute myocardial infarction
CARD_G1_list=list(pusdig[pusdig.CLIN_CD.isin(['I410','I411','I210','I211','I212','I213','I214','I215','I216','I217','I218','I219','I418'])].EVENT_ID)
#Acute pericarditis
CARD_G2_list=list(pusdig[pusdig.CLIN_CD.isin(['I420','I308','I309'])].EVENT_ID)
#Acute myocarditis
CARD_G3_list=list(pusdig[pusdig.CLIN_CD.isin(['I409','I408','I514','I401','I402','I403','I404','I405','I406','I407'
,'I412','I413','I414','I415','I416','I417','I419','I421','I422'])].EVENT_ID)

CARD_G4_list=list(pusdig[pusdig.CLIN_CD.isin(['I429','I428','I430','I516','I517','I518','I519','I255','I256'])].EVENT_ID)
CARD_G5_list=list(pusdig[pusdig.CLIN_CD.isin(['I500','I501','I509'])].EVENT_ID)
CARD_list=CARD_G1_list+CARD_G2_list+CARD_G3_list+CARD_G4_list+CARD_G5_list

MENI_list=list(pusdig[pusdig.CLIN_CD.isin(['A870','A89','G020','G028','G030','G038','G052','G0331','G039','G04','G05','G934','G312','G92'])].EVENT_ID)
RAMS_list=list(pusdig[pusdig.CLIN_CD.isin(['B022','G510','H490','H491','H492','H493','H494','H510','G511','G519'])].EVENT_ID)

MEDI_G1_list=list(pusdig[pusdig.CLIN_CD.isin(['L0312','L038','L039','M7961','M7962','M7963','M7964','M7968',
                                              'M7969','M7989','T801','T802','T888','I889','I890','I891','L049','R599'])].EVENT_ID)
MEDI_G2_list=list(pusdig[pusdig.CLIN_CD.isin(['T783','T887','T784','L500','L501','L508','L509','T749','T782','T805',
                                              'L0310','L508','R223','R229','L024','L028','L029','L042','L043','L048','L049','L299'])].EVENT_ID)
MEDI_list=MEDI_G1_list+MEDI_G2_list

##add sensitivity disease
WHOC_list=list(pusdig[pusdig.CLIN_CD.isin(['A370','A371','A378','A379'])].EVENT_ID)

APPE_list=list(pusdig[pusdig.CLIN_CD.isin(['K350','K351','K359','K36','K37','K380','K381','K382','K383','K388','K389'])].EVENT_ID)

HERN_list=list(pusdig[pusdig.CLIN_CD.isin(['K4000','K4001','K4010','K4011','K4020','K4021','K4030','K4031','K4040','K4041','K4090','K4091','K410','K411','K412','K413','K414',
'K419','K420','K421','K429','K430','K431','K439','K440','K441','K449','K450','K451','K458','K460','K461','K469'])].EVENT_ID)

DIVE_list=list(pusdig[pusdig.CLIN_CD.isin(['K5700','K5701','K5702','K5703','K5710','K5711','K5712','K513','K5720','K5721','K5722','K5723','K5730','K5731',
'K5732','K5733','K5740','K5741','K5742','K5743','K5780','K5781, K5782, K5783','K5790','K57921','K5792','K5793'])].EVENT_ID)

FEMF_list=list(pusdig[pusdig.CLIN_CD.isin(['S7200','S7201','S7202','S7203','S7204','S7205','S7208','S7210','S7211','S722','S723','S7240','S7241','S7242','S7243','S7244','S727','S728','S729'])].EVENT_ID)

CHOC_list=list(pusdig[pusdig.CLIN_CD.isin(['K8000','K8001','K8010','K8011','K8020','K8021','K8030','K8031','K8040','K8041','K8050','K8051','K8080','K8081','K810',
                                           'K811','K818','K819','K820','K821','K822','K823','K824','K828','K829','K830','K831','K832','K833','K834','K835','K835','K838','K839','K870'])].EVENT_ID)
PAND_list=list(pusdig[pusdig.CLIN_CD.isin(['K85','K860','K861','K862','K863','K868','K869','K871','K903'])].EVENT_ID)
SEPS_list=list(pusdig[pusdig.CLIN_CD.isin(pusdig.CLIN_CD.str.startswith('A40') | pusdig.CLIN_CD.str.startswith('A41'))].EVENT_ID)
HAEM_list=list(pusdig[pusdig.CLIN_CD.isin(['O224','O872']) | pusdig.CLIN_CD.str.startswith('I84')].EVENT_ID)
RENC_list=list(pusdig[pusdig.CLIN_CD.isin(pusdig.CLIN_CD.str.startswith('N20') | pusdig.CLIN_CD.str.startswith('N21') | pusdig.CLIN_CD.str.startswith('N22') | pusdig.CLIN_CD.str.startswith('N23'))].EVENT_ID)
BURN_list=list(pusdig[pusdig.CLIN_CD.isin(pusdig.CLIN_CD.str.startswith('M6137') | pusdig.CLIN_CD.str.startswith('M6138') | pusdig.CLIN_CD.str.startswith('M6139') | pusdig.CLIN_CD.str.startswith('T2')  | pusdig.CLIN_CD.str.startswith('T3'))].EVENT_ID)
LIPO_list=list(pusdig[pusdig.CLIN_CD.isin(pusdig.CLIN_CD.str.startswith('D17'))].EVENT_ID)
EPIS_list=list(pusdig[pusdig.CLIN_CD.isin(['R040'])].EVENT_ID)
CANC_list=list(can.MASTER_ENC.unique())

##Create Level1 Ethnicity group
puseve['ETHNICGP_Lvl1']=puseve['ETHNICGP']//10

puseve["COPD"] = np.where(puseve["EVENT_ID"].isin(COPD_list),1, 0)
puseve["DIAB"] = np.where(puseve["EVENT_ID"].isin(DIAB_list),1, 0)
puseve["GOUT"] = np.where(puseve["EVENT_ID"].isin(GOUT_list),1, 0)
puseve["CERE"] = np.where(puseve["EVENT_ID"].isin(CERE_list),1, 0)
puseve["CHRO"] = np.where(puseve["EVENT_ID"].isin(CHRO_list),1, 0)
puseve["ISCH"] = np.where(puseve["EVENT_ID"].isin(ISCH_list),1, 0)
puseve["SERO"] = np.where(puseve["EVENT_ID"].isin(SERO_list),1, 0)
puseve["PSOE"] = np.where(puseve["EVENT_ID"].isin(PSOE_list),1, 0)
puseve["LUPU"] = np.where(puseve["EVENT_ID"].isin(LUPU_list),1, 0)
puseve["BOWE"] = np.where(puseve["EVENT_ID"].isin(BOWE_list),1, 0)
puseve["PSOR"] = np.where(puseve["EVENT_ID"].isin(PSOR_list),1, 0)
puseve["ANKY"] = np.where(puseve["EVENT_ID"].isin(ANKY_list),1, 0)
puseve["LIVE"] = np.where(puseve["EVENT_ID"].isin(LIVE_list),1, 0)
puseve["AIDI"] = np.where(puseve["EVENT_ID"].isin(AIDI_list),1, 0)
puseve["CANC"] = np.where(puseve["EVENT_ID"].isin(CANC_list),1, 0)

print("Add disease flag for puseve patients")
psize(puseve)

puseve["STRK"] = np.where(puseve["EVENT_ID"].isin(STRK_list),1, 0)
puseve["STRK_G1"] = np.where(puseve["EVENT_ID"].isin(STRK_G1_list),1, 0)
puseve["STRK_G2"] = np.where(puseve["EVENT_ID"].isin(STRK_G2_list),1, 0)
puseve["STRK_G3"] = np.where(puseve["EVENT_ID"].isin(STRK_G3_list),1, 0)
puseve["STRK_G4"] = np.where(puseve["EVENT_ID"].isin(STRK_G4_list),1, 0)

puseve["CARD"] = np.where(puseve["EVENT_ID"].isin(CARD_list),1, 0)
puseve["CARD_G1"] = np.where(puseve["EVENT_ID"].isin(CARD_G1_list),1, 0)
puseve["CARD_G2"] = np.where(puseve["EVENT_ID"].isin(CARD_G2_list),1, 0)
puseve["CARD_G3"] = np.where(puseve["EVENT_ID"].isin(CARD_G3_list),1, 0)
puseve["CARD_G4"] = np.where(puseve["EVENT_ID"].isin(CARD_G4_list),1, 0)
puseve["CARD_G5"] = np.where(puseve["EVENT_ID"].isin(CARD_G5_list),1, 0)

puseve["MENI"] = np.where(puseve["EVENT_ID"].isin(MENI_list),1, 0)
puseve["RAMS"] = np.where(puseve["EVENT_ID"].isin(RAMS_list),1, 0)
puseve["MEDI"] = np.where(puseve["EVENT_ID"].isin(MEDI_list),1, 0)
puseve["MEDI_G1"] = np.where(puseve["EVENT_ID"].isin(MEDI_G1_list),1, 0)
puseve["MEDI_G2"] = np.where(puseve["EVENT_ID"].isin(MEDI_G2_list),1, 0)

puseve["WHOC"] = np.where(puseve["EVENT_ID"].isin(WHOC_list),1, 0)
puseve["APPE"] = np.where(puseve["EVENT_ID"].isin(APPE_list),1, 0)
puseve["HERN"] = np.where(puseve["EVENT_ID"].isin(HERN_list),1, 0)
puseve["DIVE"] = np.where(puseve["EVENT_ID"].isin(DIVE_list),1, 0)
puseve["FEMF"] = np.where(puseve["EVENT_ID"].isin(FEMF_list),1, 0)
puseve["CHOC"] = np.where(puseve["EVENT_ID"].isin(CHOC_list),1, 0)
puseve["PAND"] = np.where(puseve["EVENT_ID"].isin(PAND_list),1, 0)
puseve["SEPS"] = np.where(puseve["EVENT_ID"].isin(SEPS_list),1, 0)
puseve["HAEM"] = np.where(puseve["EVENT_ID"].isin(HAEM_list),1, 0)
puseve["RENC"] = np.where(puseve["EVENT_ID"].isin(RENC_list),1, 0)
puseve["BURN"] = np.where(puseve["EVENT_ID"].isin(BURN_list),1, 0)
puseve["LIPO"] = np.where(puseve["EVENT_ID"].isin(LIPO_list),1, 0)
puseve["EPIS"] = np.where(puseve["EVENT_ID"].isin(EPIS_list),1, 0)


print("Add high risk disease flag for puseve patients")
psize(puseve)

##add DEP Quin
puseve['DEP06_Q']=5
puseve.loc[(puseve.DEP06==2) | (puseve.DEP06==1),"DEP06_Q"]=1
puseve.loc[(puseve.DEP06==4) | (puseve.DEP06==3),"DEP06_Q"]=2
puseve.loc[(puseve.DEP06==6) | (puseve.DEP06==5),"DEP06_Q"]=3
puseve.loc[(puseve.DEP06==8) | (puseve.DEP06==7),"DEP06_Q"]=4
puseve['DEP13_Q']=5
puseve.loc[(puseve.DEP13==2) | (puseve.DEP13==1),"DEP13_Q"]=1
puseve.loc[(puseve.DEP13==4) | (puseve.DEP13==3),"DEP13_Q"]=2
puseve.loc[(puseve.DEP13==6) | (puseve.DEP13==5),"DEP13_Q"]=3
puseve.loc[(puseve.DEP13==8) | (puseve.DEP13==7),"DEP13_Q"]=4

puseve['DEP18_Q']=5
puseve.loc[(puseve.DEP18==2) | (puseve.DEP18==1),"DEP18_Q"]=1
puseve.loc[(puseve.DEP18==4) | (puseve.DEP18==3),"DEP18_Q"]=2
puseve.loc[(puseve.DEP18==6) | (puseve.DEP18==5),"DEP18_Q"]=3
puseve.loc[(puseve.DEP18==8) | (puseve.DEP18==7),"DEP18_Q"]=4

##add DHB Location
puseve.DHBDOM=puseve.DHBDOM.astype(str).str.zfill(3)
puseve=puseve.merge(DHB_Mapping,how="left",on="DHBDOM")

puseve["IMD"] = np.where(puseve["MASTER_ENC"].isin(imd_list),1, 0)

print("Add other category flag for puseve patients")
psize(puseve)


##add cancer type in the puseve dataset
for i in can_list:
  temp_list=list(can[can.MORPH==i].drop_duplicates(keep='first',subset=['MASTER_ENC']).MASTER_ENC)
  puseve[i] = np.where(puseve["MASTER_ENC"].isin(temp_list),1, 0)

print("Add cancer/which type of cancer flag for puseve patients")
psize(puseve)


puseve.loc[:,'EVSTDATE'] = pd.to_datetime(puseve['EVSTDATE'],dayfirst=True)
puseve.loc[:,'VACCINATION_DATE'] = pd.to_datetime(puseve['VACCINATION_DATE'],dayfirst=True)
puseve.loc[:,'DIAG_YEAR']=puseve['EVSTDATE'].dt.year

##Create person days
puseve['Person_Days']=puseve['EVSTDATE']-puseve['VACCINATION_DATE']
puseve['Person_Days']=puseve.Person_Days.dt.days

# puseve.loc[:,'AGE']=pd.cut(puseve['AGE_DIS'], [-1,0,9,19,29,39,49,59,64,69,74,79,150], labels=['1<','1-9','10-19','20-29','30-39','40-49','50-59','60-64','65-69','70-74','75-79','>=80'])
puseve.loc[:,'AGE']=pd.cut(puseve['AGE_DIS'], [-1,54,59,64,69,74,79,150], labels=['50-54','55-59','60-64','65-69','70-74','75-79','>=80'])
puseve_ctr_processed=puseve.copy()

lookback

In [ ]:
pusdig=pusdig_ctrlb
print('Original pusdig size')
psize(pusdig)
if Type_A_only:
  pusdig=pusdig[pusdig.DIAG_TYP=='A']

print('Type A diagnose pusdig size')
psize(pusdig)

puseve=puseve_ctrlb.drop_duplicates(['MASTER_ENC','EVENT_ID'])

##add sensitivity disease
WHOC_list=list(pusdig[pusdig.CLIN_CD.isin(['A370','A371','A378','A379'])].EVENT_ID)

APPE_list=list(pusdig[pusdig.CLIN_CD.isin(['K350','K351','K359','K36','K37','K380','K381','K382','K383','K388','K389'])].EVENT_ID)

HERN_list=list(pusdig[pusdig.CLIN_CD.isin(['K4000','K4001','K4010','K4011','K4020','K4021','K4030','K4031','K4040','K4041','K4090','K4091','K410','K411','K412','K413','K414',
'K419','K420','K421','K429','K430','K431','K439','K440','K441','K449','K450','K451','K458','K460','K461','K469'])].EVENT_ID)

DIVE_list=list(pusdig[pusdig.CLIN_CD.isin(['K5700','K5701','K5702','K5703','K5710','K5711','K5712','K513','K5720','K5721','K5722','K5723','K5730','K5731',
'K5732','K5733','K5740','K5741','K5742','K5743','K5780','K5781, K5782, K5783','K5790','K57921','K5792','K5793'])].EVENT_ID)

FEMF_list=list(pusdig[pusdig.CLIN_CD.isin(['S7200','S7201','S7202','S7203','S7204','S7205','S7208','S7210','S7211','S722','S723','S7240','S7241','S7242','S7243','S7244','S727','S728','S729'])].EVENT_ID)

CHOC_list=list(pusdig[pusdig.CLIN_CD.isin(['K8000','K8001','K8010','K8011','K8020','K8021','K8030','K8031','K8040','K8041','K8050','K8051','K8080','K8081','K810',
                                           'K811','K818','K819','K820','K821','K822','K823','K824','K828','K829','K830','K831','K832','K833','K834','K835','K835','K838','K839','K870'])].EVENT_ID)
PAND_list=list(pusdig[pusdig.CLIN_CD.isin(['K85','K860','K861','K862','K863','K868','K869','K871','K903'])].EVENT_ID)
SEPS_list=list(pusdig[pusdig.CLIN_CD.isin(pusdig.CLIN_CD.str.startswith('A40') | pusdig.CLIN_CD.str.startswith('A41'))].EVENT_ID)
HAEM_list=list(pusdig[pusdig.CLIN_CD.isin(['O224','O872']) | pusdig.CLIN_CD.str.startswith('I84')].EVENT_ID)
RENC_list=list(pusdig[pusdig.CLIN_CD.isin(pusdig.CLIN_CD.str.startswith('N20') | pusdig.CLIN_CD.str.startswith('N21') | pusdig.CLIN_CD.str.startswith('N22') | pusdig.CLIN_CD.str.startswith('N23'))].EVENT_ID)
BURN_list=list(pusdig[pusdig.CLIN_CD.isin(pusdig.CLIN_CD.str.startswith('M6137') | pusdig.CLIN_CD.str.startswith('M6138') | pusdig.CLIN_CD.str.startswith('M6139') | pusdig.CLIN_CD.str.startswith('T2')  | pusdig.CLIN_CD.str.startswith('T3'))].EVENT_ID)
LIPO_list=list(pusdig[pusdig.CLIN_CD.isin(pusdig.CLIN_CD.str.startswith('D17'))].EVENT_ID)
EPIS_list=list(pusdig[pusdig.CLIN_CD.isin(['R040'])].EVENT_ID)
CANC_list=list(can.MASTER_ENC.unique())

puseve["WHOC"] = np.where(puseve["EVENT_ID"].isin(WHOC_list),1, 0)
puseve["APPE"] = np.where(puseve["EVENT_ID"].isin(APPE_list),1, 0)
puseve["HERN"] = np.where(puseve["EVENT_ID"].isin(HERN_list),1, 0)
puseve["DIVE"] = np.where(puseve["EVENT_ID"].isin(DIVE_list),1, 0)
puseve["FEMF"] = np.where(puseve["EVENT_ID"].isin(FEMF_list),1, 0)
puseve["CHOC"] = np.where(puseve["EVENT_ID"].isin(CHOC_list),1, 0)
puseve["PAND"] = np.where(puseve["EVENT_ID"].isin(PAND_list),1, 0)
puseve["SEPS"] = np.where(puseve["EVENT_ID"].isin(SEPS_list),1, 0)
puseve["HAEM"] = np.where(puseve["EVENT_ID"].isin(HAEM_list),1, 0)
puseve["RENC"] = np.where(puseve["EVENT_ID"].isin(RENC_list),1, 0)
puseve["BURN"] = np.where(puseve["EVENT_ID"].isin(BURN_list),1, 0)
puseve["LIPO"] = np.where(puseve["EVENT_ID"].isin(LIPO_list),1, 0)
puseve["EPIS"] = np.where(puseve["EVENT_ID"].isin(EPIS_list),1, 0)


print("Add high risk disease flag for puseve patients")
psize(puseve)

puseve_ctrlb_processed=puseve.copy()

In [ ]:
puseve_hig_processed['FLAG']=1
puseve_ctr_processed['FLAG']=0
puseve_full=pd.concat([puseve_hig_processed,puseve_ctr_processed],axis=0)

data=puseve_full.copy()

data.to_csv('data_table1_typeA.csv',index=False)

table_list=['DIAG_YEAR','AGE','GENDER','ETHNICGP_Lvl1','DEP06_Q','DEP13_Q','DEP18_Q','IMD','LOCATION']

pd.set_option('display.float_format', lambda x: '%2f' % x)
data.loc[:,'EVSTDATE'] = pd.to_datetime(data['EVSTDATE'])
data['EVENPERIOD']=data.EVSTDATE.dt.to_period('M')

##data source for DHB plot
data.pivot_table(index="LOCATION",columns='FLAG',values='MASTER_ENC',aggfunc=np.count_nonzero).to_csv('location.csv')
data.pivot_table(index=["LOCATION","GENDER"],columns='FLAG',values='MASTER_ENC',aggfunc=np.count_nonzero).to_csv('dhb_gender.csv')

In [ ]:
dhb_plot=data.pivot_table(index="LOCATION",columns='FLAG',values='MASTER_ENC',aggfunc=np.count_nonzero).reset_index()
data.LOCATION=data.LOCATION.str.title()
data.pivot_table(index="LOCATION",columns='ETHNICGP_Lvl1',values='MASTER_ENC',aggfunc=np.count_nonzero,margins=True)

In [ ]:
def radar_plot(data,c_col):

  radar_df=data.pivot_table(index=c_col,columns='GENDER',values='MASTER_ENC',aggfunc=np.count_nonzero).reset_index()
  # radar_df[c_col]=c_col+':'+radar_df[c_col].astype(str)
  categories =[*radar_df[c_col],radar_df[c_col][0]]

  rank_m=[*radar_df['M'],radar_df['M'][0]]
  rank_f=[*radar_df['F'],radar_df['F'][0]]

  label_loc = np.linspace(start=0, stop=2* np.pi, num=len(categories))

  plt.figure(figsize=(8, 8))
  plt.subplot(polar=True)

  plt.plot(label_loc, rank_m, label='Male')
  plt.plot(label_loc, rank_f, label='Female')

  # plt.title('Sampling algorithms comparison', size=15)
  lines, labels = plt.thetagrids(np.degrees(label_loc), labels=categories)
  plt.legend(loc=1)
  plt.ylim(radar_df[['F','M']].min().min()-1,radar_df[['F','M']].max().max()+1)
  # plt.show()
  plt.savefig('radar_by_'+c_col+'.pdf',dpi=500)
  plt.show()

In [ ]:
radar_plot(data,'LOCATION')

In [ ]:
for i in table_list+disease_list:
  display(data.pivot_table(index=[i],columns='FLAG',values='MASTER_ENC',aggfunc=np.count_nonzero,margins=True))
  display(data.pivot_table(index=[i],columns='FLAG',values='MASTER_ENC',aggfunc=np.count_nonzero,margins=True).transform(lambda x: x*200/x.sum()))

In [ ]:
side_effect_list=['MASTER_ENC','FLAG','STRK', 'STRK_G1','STRK_G2', 'STRK_G3', 'STRK_G4', 'CARD', 'CARD_G1', 'CARD_G2',
       'CARD_G3', 'CARD_G4', 'CARD_G5', 'MENI', 'RAMS', 'MEDI', 'MEDI_G1','MEDI_G2']
sensitivity_list=['WHOC','APPE','HERN','DIVE','FEMF','CHOC','PAND','SEPS','HAEM','RENC','BURN','LIPO','EPIS']
date_column=['VACCINATION_DATE','EVSTDATE']

In [ ]:
puseve_full[side_effect_list+sensitivity_list].sum()

In [ ]:
puseve_full[side_effect_list+sensitivity_list].sum()

##Result - Table 2 + Table 3



In [ ]:
all_columns=True

In [ ]:
if all_columns:
  out_put=data.copy()
else:
  out_put=puseve_full[side_effect_list+sensitivity_list+date_column]

In [ ]:
op_hig,op_ctr=out_put[out_put.FLAG==1],out_put[out_put.FLAG==0]

In [ ]:
out_put=op_hig.copy()
##IF FLAG IS 1, then do the following
out_put['STRK_PT']=np.where(out_put['STRK']==0,42,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D'))
out_put['STRK_G1_PT']=np.where(out_put['STRK_G1']==0,42,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D'))
out_put['STRK_G2_PT']=np.where(out_put['STRK_G2']==0,42,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D'))
out_put['STRK_G3_PT']=np.where(out_put['STRK_G3']==0,42,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D'))
out_put['STRK_G4_PT']=np.where(out_put['STRK_G4']==0,42,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D'))

out_put['CARD_PT']=np.where(out_put['CARD']==0,42,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D'))
out_put['CARD_G1_PT']=np.where(out_put['CARD_G1']==0,42,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D'))
out_put['CARD_G2_PT']=np.where(out_put['CARD_G2']==0,42,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D'))
out_put['CARD_G3_PT']=np.where(out_put['CARD_G3']==0,42,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D'))
out_put['CARD_G4_PT']=np.where(out_put['CARD_G4']==0,42,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D'))
out_put['CARD_G5_PT']=np.where(out_put['CARD_G5']==0,42,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D'))

out_put['MENI_PT']=np.where(out_put['MENI']==0,42,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D'))

out_put['RAMS_PT']=np.where(out_put['RAMS']==0,14,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D'))

out_put['MEDI_PT']=np.where(out_put['MEDI']==0,14,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D'))
out_put['MEDI_G1_PT']=np.where(out_put['MEDI_G1']==0,14,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D'))
out_put['MEDI_G2_PT']=np.where(out_put['MEDI_G2']==0,14,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D'))

out_put['WHOC_PT']=np.where(out_put['WHOC']==0,42,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D'))
out_put['APPE_PT']=np.where(out_put['APPE']==0,42,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D'))
out_put['HERN_PT']=np.where(out_put['HERN']==0,42,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D'))
out_put['DIVE_PT']=np.where(out_put['DIVE']==0,42,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D'))
out_put['FEMF_PT']=np.where(out_put['FEMF']==0,42,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D'))
out_put['CHOC_PT']=np.where(out_put['CHOC']==0,42,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D'))
out_put['PAND_PT']=np.where(out_put['PAND']==0,42,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D'))
out_put['SEPS_PT']=np.where(out_put['SEPS']==0,42,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D'))
out_put['HAEM_PT']=np.where(out_put['HAEM']==0,42,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D'))
out_put['RENC_PT']=np.where(out_put['RENC']==0,42,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D'))
out_put['BURN_PT']=np.where(out_put['BURN']==0,42,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D'))
out_put['LIPO_PT']=np.where(out_put['LIPO']==0,42,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D'))
out_put['EPIS_PT']=np.where(out_put['EPIS']==0,42,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D'))

op_hig_pt=out_put.copy()

In [ ]:
out_put=op_ctr.copy()
#IF FLAG is 0 , then - 72 and maximum is 90
out_put['STRK_PT']=np.where(out_put['STRK']==0,90,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D')-72)
out_put['STRK_G1_PT']=np.where(out_put['STRK_G1']==0,90,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D')-72)
out_put['STRK_G2_PT']=np.where(out_put['STRK_G2']==0,90,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D')-72)
out_put['STRK_G3_PT']=np.where(out_put['STRK_G3']==0,90,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D')-72)
out_put['STRK_G4_PT']=np.where(out_put['STRK_G4']==0,90,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D')-72)

out_put['CARD_PT']=np.where(out_put['CARD']==0,90,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D')-72)
out_put['CARD_G1_PT']=np.where(out_put['CARD_G1']==0,90,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D')-72)
out_put['CARD_G2_PT']=np.where(out_put['CARD_G2']==0,90,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D')-72)
out_put['CARD_G3_PT']=np.where(out_put['CARD_G3']==0,90,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D')-72)
out_put['CARD_G4_PT']=np.where(out_put['CARD_G4']==0,90,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D')-72)
out_put['CARD_G5_PT']=np.where(out_put['CARD_G5']==0,90,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D')-72)

out_put['MENI_PT']=np.where(out_put['MENI']==0,90,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D')-72)

out_put['RAMS_PT']=np.where(out_put['RAMS']==0,14,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D')-72)

out_put['MEDI_PT']=np.where(out_put['MEDI']==0,14,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D')-72)
out_put['MEDI_G1_PT']=np.where(out_put['MEDI_G1']==0,14,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D')-72)
out_put['MEDI_G2_PT']=np.where(out_put['MEDI_G2']==0,14,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D')-72)

out_put['WHOC_PT']=np.where(out_put['WHOC']==0,90,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D')-72)
out_put['APPE_PT']=np.where(out_put['APPE']==0,90,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D')-72)
out_put['HERN_PT']=np.where(out_put['HERN']==0,90,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D')-72)
out_put['DIVE_PT']=np.where(out_put['DIVE']==0,90,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D')-72)
out_put['FEMF_PT']=np.where(out_put['FEMF']==0,90,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D')-72)
out_put['CHOC_PT']=np.where(out_put['CHOC']==0,90,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D')-72)
out_put['PAND_PT']=np.where(out_put['PAND']==0,90,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D')-72)
out_put['SEPS_PT']=np.where(out_put['SEPS']==0,90,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D')-72)
out_put['HAEM_PT']=np.where(out_put['HAEM']==0,90,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D')-72)
out_put['RENC_PT']=np.where(out_put['RENC']==0,90,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D')-72)
out_put['BURN_PT']=np.where(out_put['BURN']==0,90,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D')-72)
out_put['LIPO_PT']=np.where(out_put['LIPO']==0,90,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D')-72)
out_put['EPIS_PT']=np.where(out_put['EPIS']==0,90,(out_put.EVSTDATE-out_put.VACCINATION_DATE)/ np.timedelta64(1, 'D')-72)


op_ctr_pt=out_put.copy()

In [ ]:
pd.concat([op_hig_pt,op_ctr_pt]).to_csv('person_time.csv',index=False)

In [ ]:
# for i in out_put.columns[out_put.columns.str.contains('_PT')]:
#   out_put[i]=np.where(out_put[i]>90,90,out_put[i])

In [ ]:
out_put.to_csv("person_time_TypeA.csv",index=False)

Table 2 and Table 3

In [ ]:
out_put.groupby(['FLAG']).sum().T

##Statistical analysis - optional

In [ ]:
!pip install statannotations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
data.head()

In [ ]:
import seaborn as sns

from statannotations.Annotator import Annotator

plt.figure(figsize=(16,8))

x = "GENDER"
y = "AGE_DIS"
order = ['M','F']

ax = sns.boxplot(data=data, x=x, y=y)
pairs=[('M','F')]

annotator = Annotator(ax, pairs, data=data, x=x, y=y)
annotator.configure(test='Mann-Whitney', text_format='star', loc='outside')
annotator.apply_and_annotate()

plt.savefig('AGE_DIS_By_Gender.pdf', dpi=500, bbox_inches='tight')

In [ ]:
plt.figure(figsize=(16,8))

x = "GENDER"
y = "AGE_DIS"
hue= "AGE"
order = ['M','F']

pairs=[
    (("M", '50-64'), ("F", '50-64')),
    (("M",'65-69'), ("F", '65-69')),
    (("M", '70-74'), ("F", '70-74')),
    (("M", '75-79'), ("F", '75-79')),
    (("M", '>=80'), ("F", '>=80')),
    ]

# In this case we just have to pass the list of categories objects to the annotate function.
age_bucket_list = data['AGE'].unique()


ax = sns.boxplot(data=data, x=x, y=y, order=order, hue=hue)
annot = Annotator(ax, pairs, data=data, x=x, y=y, order=order, hue=hue)
# annot.new_plot(ax, pairs, data=data, x=x, y=y, order=order, hue=hue)
annot.configure(test='Mann-Whitney', verbose=2)
annot.apply_test()
annot.annotate()
plt.legend(loc='upper left', bbox_to_anchor=(1.03, 1))
plt.savefig('AGE_BUCKET_By_Gender.pdf', dpi=500, bbox_inches='tight')

In [ ]:
plt.figure(figsize=(16,8))

x = "GENDER"
y = "AGE_DIS"
hue= "AGE"
order = ['M','F']

pairs=[
    (("M", '50-64'), ("F", '50-64')),
    (("M",'65-69'), ("F", '65-69')),
    (("M", '70-74'), ("F", '70-74')),
    (("M", '75-79'), ("F", '75-79')),
    (("M", '>=80'), ("F", '>=80')),
    ]

# In this case we just have to pass the list of categories objects to the annotate function.
age_bucket_list = data['AGE'].unique()


ax = sns.violinplot(data=data, x=x, y=y, order=order, hue=hue)
annot = Annotator(ax, pairs, data=data, x=x, y=y, order=order, plot="violinplot",hue=hue)
# annot.new_plot(ax, pairs, data=data, x=x, y=y, order=order, hue=hue)
annot.configure(test='Mann-Whitney', verbose=2)
annot.apply_test()
annot.annotate()
plt.legend(loc='upper left', bbox_to_anchor=(1.03, 1))
plt.savefig('AGE_BUCKET_By_Gender_Violin.pdf', dpi=500, bbox_inches='tight')

In [ ]:
plt.figure(figsize=(16,8))

y = "GENDER"
x= "AGE_DIS"
hue= "AGE"
order = ['M','F']

pairs=[
    (("M", '50-64'), ("F", '50-64')),
    (("M",'65-69'), ("F", '65-69')),
    (("M", '70-74'), ("F", '70-74')),
    (("M", '75-79'), ("F", '75-79')),
    (("M", '>=80'), ("F", '>=80')),
    ]

# In this case we just have to pass the list of categories objects to the annotate function.
age_bucket_list = data['AGE'].unique()

configuration = {'test':'Mann-Whitney',
                 'comparisons_correction':None,
                 'text_format':'star'}

ax = sns.stripplot(data=data, x=x, y=y,orient='h', order=order, hue=hue)
annot = Annotator(ax, pairs, data=data, x=x, y=y, order=order, plot='swarmplot', orient='h',hue=hue)
# annot.new_plot(ax, pairs, data=data, x=x, y=y, order=order, hue=hue)
# annot.configure(test='Mann-Whitney', verbose=2)
annot.configure(**{**configuration, "text_format":"simple", "text_offset":6})\
    .apply_test().annotate()
plt.legend(loc='upper left', bbox_to_anchor=(1.03, 1))
plt.savefig('AGE_BUCKET_By_Gender_swarm.pdf', dpi=500, bbox_inches='tight')

In [ ]:
########FINISHED#######################